# LISS Panel Data Loading

This notebook loads all CSV files from the current directory.


In [11]:
# Cell 1:  Load All CSV Files
import pandas as pd
import os
from pathlib import Path
import glob

# Get current directory
current_dir = Path('.')

# Find all CSV files in current directory (excluding subdirectories)
csv_files = list(current_dir.glob('*.csv'))

# Exclude intermediate result files (from previous runs)
excluded_files = [
    'merged_liss_data.csv',
    'LISS_Final_Features_X.csv',
    'LISS_Final_Features_X_with_ID.csv',
    'LISS_Final_Features_Y.csv'
]

csv_files = [f for f in csv_files if f.name not in excluded_files]

print(f"Found {len(csv_files)} CSV files (excluding intermediate results):")
for f in csv_files:
    print(f"  - {f.name}")

if excluded_files:
    print(f"\nExcluded intermediate result files ({len(excluded_files)}):")
    for fname in excluded_files:
        print(f"  - {fname}")

# Dictionary to store all dataframes
dataframes = {}

# Load each CSV file
for csv_file in csv_files:
    # Use filename without extension as key
    key = csv_file.stem
    print(f"\nLoading {csv_file.name}...")
    
    # Try different combinations of separator and encoding
    loaded = False
    
    # First, try to detect separator by reading first line
    try:
        with open(csv_file, 'r', encoding='utf-8') as f:
            first_line = f.readline()
            # Check if semicolon is used (common in European CSV files)
            if ';' in first_line and first_line.count(';') > first_line.count(','):
                sep = ';'
            else:
                sep = ','
    except:
        sep = ','  # Default to comma
    
    # Try different encodings and separators
    for encoding in ['utf-8', 'latin-1']:
        for separator in [sep, ';', ',']:
            try:
                df = pd.read_csv(csv_file, encoding=encoding, sep=separator, low_memory=False)
                # Check if we got a reasonable number of columns (more than 1)
                if df.shape[1] > 1:
                    dataframes[key] = df
                    sep_info = f" (sep='{separator}', encoding='{encoding}')" if separator != ',' or encoding != 'utf-8' else ""
                    print(f"  [OK] Loaded successfully{sep_info}: {df.shape[0]} rows, {df.shape[1]} columns")
                    loaded = True
                    break
                else:
                    # If only 1 column, separator might be wrong
                    continue
            except Exception as e:
                continue
        
        if loaded:
            break
    
    if not loaded:
        print(f"  [ERROR] Error: Could not load {csv_file.name} with any combination of separator and encoding")

print(f"\n{'='*60}")
print(f"Summary: Successfully loaded {len(dataframes)} dataframes")
print(f"{'='*60}")

# Display summary of loaded dataframes
for key, df in dataframes.items():
    print(f"\n{key}:")
    print(f"  Shape: {df.shape}")
    print(f"  Columns: {list(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")


Found 28 CSV files (excluding intermediate results):
  - Family and Household wave 16_23p_EN_1.0p.csv
  - SHAP_Feature_Importance.csv
  - social interation and leisure wave 16_23p_EN_1.0p.csv
  - LISS_Trimmed_76_Features.csv
  - Health wave 16_23p_EN_1.0p.csv
  - Background Variables wave 16_202307_EN_1.0p.csv
  - Work and Schooling wave 17_24q_EN_1.0p.csv
  - LISS_Final_Y_M2.csv
  - LISS_Final_Y_M3.csv
  - LISS_Final_Y_M1.csv
  - Personality wave 16_24p_EN_1.0p.csv
  - income wave 16_23p_EN_1.0p.csv
  - LISS_Final_Y_M4.csv
  - LISS_Final_Y_M5.csv
  - RQ1_Incremental_Contribution_Analysis.csv
  - Rubins_Rule_SHAP_Importance_Detailed.csv
  - income wave 17_24q_EN_1.0p.csv
  - Health wave 17_24q_EN_1.0p.csv
  - LISS_Final_X_M1.csv
  - LISS_Final_X_M3.csv
  - social interation and leisure wave 17_24q_EN_1.0p.csv
  - LISS_Final_X_M2.csv
  - Work and Schooling wave 16_23p_EN_1.0p.csv
  - Rubins_Rule_SHAP_Importance.csv
  - Subgroup_Error_Analysis_Results.csv
  - LISS_Final_X_M5.csv
  - LISS

In [12]:
# Cell 2:  Merge All DataFrames
# Merge all dataframes using nomem_encr as join key
print("="*60)
print("Merging all dataframes using 'nomem_encr' as join key")
print("="*60)

# Check which dataframes have nomem_encr column
dataframes_with_key = {}
dataframes_without_key = {}

for key, df in dataframes.items():
    if 'nomem_encr' in df.columns:
        dataframes_with_key[key] = df
        print(f"[OK] {key}: has 'nomem_encr' column ({df.shape[0]} rows, {df.shape[1]} columns)")
    else:
        dataframes_without_key[key] = df
        print(f"[WARNING] {key}: missing 'nomem_encr' column - will be skipped")

if len(dataframes_without_key) > 0:
    print(f"\nWarning: {len(dataframes_without_key)} dataframes will be skipped due to missing 'nomem_encr' column")

if len(dataframes_with_key) == 0:
    print("\n[ERROR] Error: No dataframes found with 'nomem_encr' column!")
    print("Cannot proceed with merge operation.")
else:
    print(f"\n{'='*60}")
    print(f"Starting merge of {len(dataframes_with_key)} dataframes...")
    print(f"{'='*60}")
    
    # Start with the first dataframe
    merged_df = None
    merge_order = []
    
    for i, (key, df) in enumerate(dataframes_with_key.items()):
        if merged_df is None:
            # Initialize with first dataframe
            merged_df = df.copy()
            merge_order.append(key)
            print(f"\n[1/{len(dataframes_with_key)}] Starting with: {key}")
            print(f"    Initial shape: {merged_df.shape}")
        else:
            # Merge with next dataframe
            print(f"\n[{i+1}/{len(dataframes_with_key)}] Merging: {key}")
            print(f"    Shape before merge: {merged_df.shape}")
            print(f"    Shape of new dataframe: {df.shape}")
            
            # Check for overlapping columns (excluding nomem_encr)
            overlapping_cols = set(merged_df.columns) & set(df.columns) - {'nomem_encr'}
            
            if overlapping_cols:
                # Use suffixes for overlapping columns
                merged_df = pd.merge(
                    merged_df, 
                    df, 
                    on='nomem_encr', 
                    how='outer',
                    suffixes=('', f'_{key}')
                )
            else:
                # No overlapping columns, simple merge
                merged_df = pd.merge(
                    merged_df, 
                    df, 
                    on='nomem_encr', 
                    how='outer'
                )
            
            merge_order.append(key)
            print(f"    Shape after merge: {merged_df.shape}")
            print(f"    Unique respondents (nomem_encr): {merged_df['nomem_encr'].nunique()}")
    
    print(f"\n{'='*60}")
    print("Merge completed successfully!")
    print(f"{'='*60}")
    print(f"\nFinal merged dataset:")
    print(f"  Total rows: {len(merged_df):,}")
    print(f"  Total columns: {len(merged_df.columns):,}")
    print(f"  Unique respondents (nomem_encr): {merged_df['nomem_encr'].nunique():,}")
    print(f"\nMerge order: {' -> '.join(merge_order)}")
    
    # Display column information
    print(f"\nColumn overview (first 20 columns):")
    for i, col in enumerate(merged_df.columns[:20], 1):
        print(f"  {i:2d}. {col}")
    if len(merged_df.columns) > 20:
        print(f"  ... and {len(merged_df.columns) - 20} more columns")
    
    # Save merged dataset
    output_file = 'merged_liss_data.csv'
    print(f"\n{'='*60}")
    print(f"Saving merged dataset to: {output_file}")
    print(f"{'='*60}")
    
    try:
        merged_df.to_csv(output_file, index=False, encoding='utf-8')
        file_size = os.path.getsize(output_file) / (1024 * 1024)  # Size in MB
        print(f"[OK] Successfully saved to '{output_file}'")
        print(f"  File size: {file_size:.2f} MB")
    except Exception as e:
        print(f"[ERROR] Error saving file: {e}")
        # Try with different encoding
        try:
            merged_df.to_csv(output_file, index=False, encoding='latin-1')
            file_size = os.path.getsize(output_file) / (1024 * 1024)
            print(f"[OK] Successfully saved with latin-1 encoding")
            print(f"  File size: {file_size:.2f} MB")
        except Exception as e2:
            print(f"[ERROR] Error with latin-1 encoding: {e2}")
    
    print(f"\n{'='*60}")
    print("Merged dataset is available as 'merged_df' variable")
    print(f"{'='*60}")


Merging all dataframes using 'nomem_encr' as join key
[OK] Family and Household wave 16_23p_EN_1.0p: has 'nomem_encr' column (5206 rows, 402 columns)
[WARNING] SHAP_Feature_Importance: missing 'nomem_encr' column - will be skipped
[OK] social interation and leisure wave 16_23p_EN_1.0p: has 'nomem_encr' column (5991 rows, 450 columns)
[OK] LISS_Trimmed_76_Features: has 'nomem_encr' column (12939 rows, 77 columns)
[OK] Health wave 16_23p_EN_1.0p: has 'nomem_encr' column (6058 rows, 209 columns)
[OK] Background Variables wave 16_202307_EN_1.0p: has 'nomem_encr' column (8626 rows, 34 columns)
[OK] Work and Schooling wave 17_24q_EN_1.0p: has 'nomem_encr' column (5792 rows, 422 columns)
[WARNING] LISS_Final_Y_M2: missing 'nomem_encr' column - will be skipped
[WARNING] LISS_Final_Y_M3: missing 'nomem_encr' column - will be skipped
[WARNING] LISS_Final_Y_M1: missing 'nomem_encr' column - will be skipped
[OK] Personality wave 16_24p_EN_1.0p: has 'nomem_encr' column (5477 rows, 159 columns)
[OK]

In [13]:
# Cell 3:  Preliminary Feature Reduction (Pre-MICE Trimming)
print("="*60)
print("Preliminary Feature Reduction (Pre-MICE Trimming)")
print("="*60)

if 'merged_df' in globals() and merged_df is not None:
    print(f"\nOriginal dataset shape: {merged_df.shape}")
    print(f"  Rows: {merged_df.shape[0]:,}")
    print(f"  Columns: {merged_df.shape[1]:,}")
    
    # ============================================================================
    # Task 1: Define Core Variables to Keep (Keep List)
    # ============================================================================
    print("\n" + "="*60)
    print("Task 1: Defining Core Variables to Keep")
    print("="*60)
    
    columns_to_keep = []
    
    # 1. All Big Five original items: cp24p020 to cp24p069 (50 items)
    big_five_items = [f'cp24p{i:03d}' for i in range(20, 70)]
    columns_to_keep.extend(big_five_items)
    print(f"\n1. Big Five original items: {len(big_five_items)} variables")
    print(f"   Range: cp24p020 to cp24p069")
    
    # 2. All core Delta variables' original wave data
    core_delta_vars = [
        'cw23p133', 'cw24q133',  # Job Satisfaction
        'cw23p429', 'cw24q429',  # Autonomy
        'cw23p428', 'cw24q428',  # Workload
        'cw23p430', 'cw24q430',  # Skills
        'cw23p610', 'cw24q610'   # Remote Hours
    ]
    columns_to_keep.extend(core_delta_vars)
    print(f"\n2. Core Delta variables' original wave data: {len(core_delta_vars)} variables")
    print(f"   Variables: {', '.join(core_delta_vars)}")
    
    # 3. All WLB/Health Delta variables' original wave data
    wlb_health_delta_vars = [
        'cw23p391', 'cw24q391',  # WFC Proxy
        'ch23p022', 'ch24q022'   # Health Hindrance
    ]
    columns_to_keep.extend(wlb_health_delta_vars)
    print(f"\n3. WLB/Health Delta variables' original wave data: {len(wlb_health_delta_vars)} variables")
    print(f"   Variables: {', '.join(wlb_health_delta_vars)}")
    
    # 4. All baseline control variables
    baseline_control_vars = [
        'nomem_encr',      # ID
        'cw23p003',        # Age
        'cf23p455',        # Children Count
        'ch23p004',        # Self-rated Health
        'ch23p001',        # Gender
        'ci23p337',        # Income (W16)
        'cw23p005'         # Education
    ]
    columns_to_keep.extend(baseline_control_vars)
    print(f"\n4. Baseline control variables: {len(baseline_control_vars)} variables")
    print(f"   Variables: {', '.join(baseline_control_vars)}")
    
    # 5. All W16/W17 income and occupation variables (for MICE)
    income_occupation_vars = [
        'ci23p337',        # W16 Income
        'ci24q337',        # W17 Income
        'ci23p383',        # W16 Occupation
        'cw23p525',        # W16 Main Occupation
        'ci24q383',        # W17 Occupation
        'cw24q525'         # W17 Main Occupation
    ]
    columns_to_keep.extend(income_occupation_vars)
    print(f"\n5. W16/W17 income and occupation variables (for MICE): {len(income_occupation_vars)} variables")
    print(f"   Variables: {', '.join(income_occupation_vars)}")
    
    # Remove duplicates while preserving order
    columns_to_keep = list(dict.fromkeys(columns_to_keep))
    
    print(f"\n" + "-"*60)
    print(f"Total unique variables to keep: {len(columns_to_keep)}")
    print(f"-"*60)
    
    # ============================================================================
    # Task 2: Execute Filtering
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Executing Feature Filtering")
    print("="*60)
    
    # Check which columns exist in the dataset
    available_columns = [col for col in columns_to_keep if col in merged_df.columns]
    missing_columns = [col for col in columns_to_keep if col not in merged_df.columns]
    
    print(f"\nColumn availability check:")
    print(f"  Available columns: {len(available_columns)}/{len(columns_to_keep)}")
    if missing_columns:
        print(f"  Missing columns: {len(missing_columns)}")
        print(f"    {', '.join(missing_columns[:10])}{'...' if len(missing_columns) > 10 else ''}")
    
    # Create new DataFrame with only selected columns
    merged_df_trimmed = merged_df[available_columns].copy()
    
    print(f"\n[OK] Created trimmed dataset with {len(available_columns)} columns")
    
    # ============================================================================
    # Task 3: Output Validation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: Output Validation")
    print("="*60)
    
    print(f"\nTrimmed dataset shape:")
    print(f"  Rows: {merged_df_trimmed.shape[0]:,}")
    print(f"  Columns: {merged_df_trimmed.shape[1]:,}")
    
    reduction_ratio = (1 - merged_df_trimmed.shape[1] / merged_df.shape[1]) * 100
    print(f"\nFeature reduction:")
    print(f"  Original columns: {merged_df.shape[1]:,}")
    print(f"  Trimmed columns: {merged_df_trimmed.shape[1]:,}")
    print(f"  Reduction: {reduction_ratio:.1f}%")
    
    print(f"\n" + "="*60)
    print("All Column Names in Trimmed Dataset")
    print("="*60)
    print(f"\nTotal: {len(merged_df_trimmed.columns)} columns\n")
    
    # Group columns by category for better readability
    print("Column names by category:")
    print("-"*60)
    
    # Big Five items
    big_five_in_dataset = [col for col in big_five_items if col in merged_df_trimmed.columns]
    if big_five_in_dataset:
        print(f"\n1. Big Five Items ({len(big_five_in_dataset)} columns):")
        for i, col in enumerate(big_five_in_dataset, 1):
            print(f"   {i:2d}. {col}")
    
    # Core Delta variables
    core_delta_in_dataset = [col for col in core_delta_vars if col in merged_df_trimmed.columns]
    if core_delta_in_dataset:
        print(f"\n2. Core Delta Variables ({len(core_delta_in_dataset)} columns):")
        for i, col in enumerate(core_delta_in_dataset, 1):
            print(f"   {i:2d}. {col}")
    
    # WLB/Health Delta variables
    wlb_health_in_dataset = [col for col in wlb_health_delta_vars if col in merged_df_trimmed.columns]
    if wlb_health_in_dataset:
        print(f"\n3. WLB/Health Delta Variables ({len(wlb_health_in_dataset)} columns):")
        for i, col in enumerate(wlb_health_in_dataset, 1):
            print(f"   {i:2d}. {col}")
    
    # Baseline control variables
    baseline_in_dataset = [col for col in baseline_control_vars if col in merged_df_trimmed.columns]
    if baseline_in_dataset:
        print(f"\n4. Baseline Control Variables ({len(baseline_in_dataset)} columns):")
        for i, col in enumerate(baseline_in_dataset, 1):
            print(f"   {i:2d}. {col}")
    
    # Income and occupation variables
    income_occ_in_dataset = [col for col in income_occupation_vars if col in merged_df_trimmed.columns]
    if income_occ_in_dataset:
        print(f"\n5. Income and Occupation Variables ({len(income_occ_in_dataset)} columns):")
        for i, col in enumerate(income_occ_in_dataset, 1):
            print(f"   {i:2d}. {col}")
    
    # Update global variable
    merged_df = merged_df_trimmed
    
    print(f"\n" + "="*60)
    print("[OK] Pre-MICE trimming completed!")
    print(f"Trimmed dataset is available as 'merged_df' variable")
    print(f"Expected column count: 70-80 columns")
    print(f"Actual column count: {merged_df_trimmed.shape[1]} columns")
    print("="*60)
    
    # ============================================================================
    # Save Trimmed Dataset to Disk
    # ============================================================================
    print("\n" + "="*60)
    print("Saving Trimmed Dataset to Disk")
    print("="*60)
    
    # Define filename
    output_filename = 'LISS_Trimmed_76_Features.csv'
    
    print(f"\nSaving to: {output_filename}")
    print(f"  Rows: {merged_df_trimmed.shape[0]:,}")
    print(f"  Columns: {merged_df_trimmed.shape[1]:,}")
    
    # Save to CSV with index
    merged_df_trimmed.to_csv(output_filename, index=True, encoding='utf-8')
    
    # Check file size
    import os
    if os.path.exists(output_filename):
        file_size = os.path.getsize(output_filename) / (1024 * 1024)  # Size in MB
        print(f"\n[OK] Successfully saved to '{output_filename}'")
        print(f"  File size: {file_size:.2f} MB")
    else:
        print(f"\n[ERROR] Error: File was not created successfully")
    
    print("="*60)
    
else:
    print("\n[ERROR] Error: 'merged_df' not found. Please run the merge cell (Cell 2) first.")


Preliminary Feature Reduction (Pre-MICE Trimming)

Original dataset shape: (12939, 3283)
  Rows: 12,939
  Columns: 3,283

Task 1: Defining Core Variables to Keep

1. Big Five original items: 50 variables
   Range: cp24p020 to cp24p069

2. Core Delta variables' original wave data: 10 variables
   Variables: cw23p133, cw24q133, cw23p429, cw24q429, cw23p428, cw24q428, cw23p430, cw24q430, cw23p610, cw24q610

3. WLB/Health Delta variables' original wave data: 4 variables
   Variables: cw23p391, cw24q391, ch23p022, ch24q022

4. Baseline control variables: 7 variables
   Variables: nomem_encr, cw23p003, cf23p455, ch23p004, ch23p001, ci23p337, cw23p005

5. W16/W17 income and occupation variables (for MICE): 6 variables
   Variables: ci23p337, ci24q337, ci23p383, cw23p525, ci24q383, cw24q525

------------------------------------------------------------
Total unique variables to keep: 76
------------------------------------------------------------

Task 2: Executing Feature Filtering

Column ava

In [14]:
# Cell 4:  LISS Missing Value Standardization
# LISS Missing Value Standardization
import numpy as np

print("="*60)
print("LISS Missing Value Standardization")
print("="*60)

# 1. Create mapping table for LISS missing value codes
LISS_MISSING_CODES = {
    -9: "I don't know (通用)",
    -8: "I prefer not to say (通用)",
    -13: "I don't know (收入特定)",
    -14: "Prefer not to say (收入特定)",
    999: "I don't know (评分特定, 0-10量表)",
    99: "I don't know (分类特定)",
    9999: "I don't know (大范围整数)"
}

print("\nLISS Missing Value Codes Mapping:")
for code, description in LISS_MISSING_CODES.items():
    print(f"  {code:5d}: {description}")

def standardize_liss_missing_values(df, missing_codes=None):
    """
    Standardize LISS missing value codes to NaN.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe
    missing_codes : dict, optional
        Dictionary mapping codes to descriptions. If None, uses default LISS codes.
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with missing values standardized
    dict
        Statistics about missing values before and after processing
    """
    if missing_codes is None:
        missing_codes = LISS_MISSING_CODES
    
    # Create a copy to avoid modifying original
    df_processed = df.copy()
    
    # Store statistics
    stats = {
        'before': {},
        'after': {},
        'replaced': {}
    }
    
    # Get all columns except nomem_encr (identifier)
    columns_to_process = [col for col in df_processed.columns if col != 'nomem_encr']
    
    print(f"\nProcessing {len(columns_to_process)} columns...")
    
    # Calculate missing values before processing
    for col in columns_to_process:
        stats['before'][col] = df_processed[col].isna().sum()
    
    # Replace missing value codes with NaN
    codes_to_replace = list(missing_codes.keys())
    print(f"\nReplacing missing value codes: {codes_to_replace}")
    
    for col in columns_to_process:
        # Count how many values will be replaced
        replaced_count = 0
        for code in codes_to_replace:
            mask = df_processed[col] == code
            replaced_count += mask.sum()
            df_processed.loc[mask, col] = np.nan
        
        if replaced_count > 0:
            stats['replaced'][col] = replaced_count
    
    # Calculate missing values after processing
    for col in columns_to_process:
        stats['after'][col] = df_processed[col].isna().sum()
    
    return df_processed, stats

def handle_income_zero_values(df):
    """
    Handle special case of income = 0 values.
    
    Logic:
    - If income = 0 AND main occupation shows paid work (codes 1-3), 
      then treat 0 as missing value
    - Otherwise, keep 0 (genuine zero income)
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with income zeros handled
    dict
        Statistics about income zero handling
    """
    df_processed = df.copy()
    income_stats = {}
    
    # Income variables to check (Wave 16 and Wave 17)
    income_vars = {
        'ci23p337': 'ci23p383',  # W16 income -> W16 main occupation
        'ci24q337': 'cw24q525',  # W17 income -> W17 main occupation (may need adjustment)
    }
    
    # Also check for alternative occupation variable names
    alt_occupation_vars = {
        'ci23p337': ['ci23p383', 'cw23p525'],  # W16 alternatives
        'ci24q337': ['cw24q525', 'ci24q383'],  # W17 alternatives
    }
    
    print("\n" + "="*60)
    print("Handling Income Zero Values")
    print("="*60)
    
    for income_var, occupation_var in income_vars.items():
        if income_var not in df_processed.columns:
            continue
        
        # Find alternative occupation variable if primary doesn't exist
        if occupation_var not in df_processed.columns:
            if income_var in alt_occupation_vars:
                for alt_var in alt_occupation_vars[income_var]:
                    if alt_var in df_processed.columns:
                        occupation_var = alt_var
                        break
        
        if occupation_var not in df_processed.columns:
            print(f"\n[WARNING] {income_var}: Occupation variable not found, skipping zero handling")
            continue
        
        # Count zeros before processing
        zeros_before = (df_processed[income_var] == 0).sum()
        
        # Find rows where income = 0 AND occupation indicates paid work (1, 2, or 3)
        # Note: Adjust codes based on actual LISS codebook
        paid_work_mask = df_processed[occupation_var].isin([1, 2, 3])
        income_zero_mask = (df_processed[income_var] == 0)
        zero_with_paid_work = income_zero_mask & paid_work_mask
        
        # Replace zeros with NaN where individual has paid work
        n_replaced = zero_with_paid_work.sum()
        df_processed.loc[zero_with_paid_work, income_var] = np.nan
        
        zeros_after = (df_processed[income_var] == 0).sum()
        
        income_stats[income_var] = {
            'zeros_before': zeros_before,
            'zeros_after': zeros_after,
            'replaced_to_na': n_replaced,
            'occupation_var_used': occupation_var
        }
        
        print(f"\n{income_var}:")
        print(f"  Zeros before: {zeros_before}")
        print(f"  Zeros with paid work (replaced): {n_replaced}")
        print(f"  Zeros after: {zeros_after}")
        print(f"  Occupation variable: {occupation_var}")
    
    return df_processed, income_stats

# 2. Apply missing value standardization
if 'merged_df' in globals() and merged_df is not None:
    print("\n" + "="*60)
    print("Step 1: Standardizing LISS Missing Value Codes")
    print("="*60)
    
    # Standardize missing values
    merged_df_cleaned, missing_stats = standardize_liss_missing_values(merged_df)
    
    print("\n" + "="*60)
    print("Step 2: Handling Income Zero Values")
    print("="*60)
    
    # Handle income zero values
    merged_df_cleaned, income_stats = handle_income_zero_values(merged_df_cleaned)
    
    # 3. Generate summary statistics
    print("\n" + "="*60)
    print("Missing Value Processing Summary")
    print("="*60)
    
    # Calculate overall statistics
    total_cols = len([c for c in merged_df_cleaned.columns if c != 'nomem_encr'])
    cols_with_missing = sum(1 for col in merged_df_cleaned.columns 
                           if col != 'nomem_encr' and merged_df_cleaned[col].isna().any())
    
    total_missing_before = sum(missing_stats['before'].values())
    total_missing_after = merged_df_cleaned.isna().sum().sum()
    total_replaced = sum(missing_stats['replaced'].values())
    
    print(f"\nOverall Statistics:")
    print(f"  Total columns processed: {total_cols}")
    print(f"  Columns with missing values: {cols_with_missing}")
    print(f"  Total missing values before: {total_missing_before:,}")
    print(f"  Total missing values after: {total_missing_after:,}")
    print(f"  Values replaced by standardization: {total_replaced:,}")
    
    # Show top 20 variables with most missing values
    print(f"\n{'='*60}")
    print("Top 20 Variables with Most Missing Values (After Processing)")
    print("="*60)
    
    missing_counts = merged_df_cleaned.isna().sum()
    missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)
    
    print(f"\n{'Variable':<40} {'Missing Count':<15} {'Missing %':<10}")
    print("-" * 65)
    
    for var, count in missing_counts.head(20).items():
        pct = (count / len(merged_df_cleaned)) * 100
        print(f"{var:<40} {count:<15,} {pct:>6.2f}%")
    
    if len(missing_counts) > 20:
        print(f"\n... and {len(missing_counts) - 20} more variables with missing values")
    
    # Show variables where values were replaced
    if missing_stats['replaced']:
        print(f"\n{'='*60}")
        print("Variables with Replaced Missing Value Codes")
        print("="*60)
        print(f"\n{'Variable':<40} {'Values Replaced':<15}")
        print("-" * 55)
        
        sorted_replaced = sorted(missing_stats['replaced'].items(), 
                                key=lambda x: x[1], reverse=True)
        for var, count in sorted_replaced[:20]:
            print(f"{var:<40} {count:<15,}")
        
        if len(sorted_replaced) > 20:
            print(f"\n... and {len(sorted_replaced) - 20} more variables")
    
    # Update the merged_df variable
    merged_df = merged_df_cleaned
    
    print(f"\n{'='*60}")
    print("[OK] Missing value processing completed!")
    print("Cleaned dataset is available as 'merged_df' variable")
    print(f"{'='*60}")
    
else:
    print("\n[ERROR] Error: 'merged_df' not found. Please run the merge cell first.")


LISS Missing Value Standardization

LISS Missing Value Codes Mapping:
     -9: I don't know (通用)
     -8: I prefer not to say (通用)
    -13: I don't know (收入特定)
    -14: Prefer not to say (收入特定)
    999: I don't know (评分特定, 0-10量表)
     99: I don't know (分类特定)
   9999: I don't know (大范围整数)

Step 1: Standardizing LISS Missing Value Codes

Processing 75 columns...

Replacing missing value codes: [-9, -8, -13, -14, 999, 99, 9999]

Step 2: Handling Income Zero Values

Handling Income Zero Values

ci23p337:
  Zeros before: 0
  Zeros with paid work (replaced): 0
  Zeros after: 0
  Occupation variable: ci23p383

ci24q337:
  Zeros before: 0
  Zeros with paid work (replaced): 0
  Zeros after: 0
  Occupation variable: cw24q525

Missing Value Processing Summary

Overall Statistics:
  Total columns processed: 75
  Columns with missing values: 75
  Total missing values before: 560,385
  Total missing values after: 560,385
  Values replaced by standardization: 0

Top 20 Variables with Most Missing Va

In [15]:
# Cell 5:  Step 4.5 - Logical Range Validation and Cleaning
print("="*60)
print("Step 4.5 - Logical Range Validation and Cleaning")
print("="*60)

import numpy as np
import pandas as pd

if 'merged_df' in globals() and merged_df is not None:
    df = merged_df.copy()
    print(f"\nOriginal merged_df shape: {df.shape}")
    
    # =====================================================================
    # Task 1: Define Logical Range Rules
    # =====================================================================
    print("\n" + "="*60)
    print("Task 1: Defining Logical Range Rules")
    print("="*60)
    
    range_rules = {}
    
    # 1. Job Satisfaction: cw23p133, cw24q133 (0–10)
    job_sat_vars = ['cw23p133', 'cw24q133']
    for col in job_sat_vars:
        range_rules[col] = (0, 10)
    print(f"Job Satisfaction variables: {job_sat_vars} -> [0, 10]")
    
    # 2. Big Five items: cp24p020–cp24p069 (1–5)
    big_five_items = [f'cp24p{i:03d}' for i in range(20, 70)]
    for col in big_five_items:
        range_rules[col] = (1, 5)
    print(f"Big Five items: cp24p020–cp24p069 ({len(big_five_items)} items) -> [1, 5]")
    
    # 3. Work characteristics (1–5 Likert)
    work_char_vars = [
        'cw23p429', 'cw24q429',  # Autonomy
        'cw23p428', 'cw24q428',  # Workload
        'cw23p430', 'cw24q430'   # Skills
    ]
    for col in work_char_vars:
        range_rules[col] = (1, 5)
    print(f"Work characteristics (1–5 Likert): {work_char_vars} -> [1, 5]")
    
    # 4. Health variables (Likert 1–5)
    health_vars = ['ch23p022', 'ch24q022', 'ch23p004', 'ch24p004']
    for col in health_vars:
        range_rules[col] = (1, 5)
    print(f"Health variables (Likert): {health_vars} -> [1, 5]")
    
    # 5. WFC binary indicator (1–2)
    wfc_vars = ['cw23p391', 'cw24q391']
    for col in wfc_vars:
        range_rules[col] = (1, 2)
    print(f"WFC binary indicators: {wfc_vars} -> [1, 2]")
    
    # 6. Remote work hours (0–168 per week)
    remote_hours_vars = ['cw23p610', 'cw24q610']
    for col in remote_hours_vars:
        range_rules[col] = (0, 168)
    print(f"Remote work hours: {remote_hours_vars} -> [0, 168]")
    
    # 7. Control variables
    control_ranges = {
        'cw23p003': (16, 100),  # Age
        'cf23p455': (0, 20)     # Children count
    }
    for col, (lo, hi) in control_ranges.items():
        range_rules[col] = (lo, hi)
    print(f"Control variables: cw23p003 -> [16, 100], cf23p455 -> [0, 20]")
    
    # =====================================================================
    # Task 2: Apply Logical Range Cleaning
    # =====================================================================
    print("\n" + "="*60)
    print("Task 2: Applying Logical Range Cleaning")
    print("="*60)
    
    cleaned_columns = []
    total_invalid_before = 0
    
    for col, (min_val, max_val) in range_rules.items():
        if col not in df.columns:
            continue
        
        # Ensure numeric dtype
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Count invalid values before cleaning
        invalid_mask = (df[col] < min_val) | (df[col] > max_val)
        n_invalid = invalid_mask.sum()
        total_invalid_before += int(n_invalid)
        
        if n_invalid > 0:
            print(f"[CLEAN] {col}: {n_invalid} values outside [{min_val}, {max_val}] -> set to NaN")
            df.loc[invalid_mask, col] = np.nan
            cleaned_columns.append(col)
    
    if cleaned_columns:
        print(f"\n[OK] Logical cleaning applied to {len(cleaned_columns)} columns")
        print(f"Total invalid values set to NaN: {total_invalid_before}")
    else:
        print("\n[INFO] No columns required logical range cleaning (no out-of-range values found)")
    
    # =====================================================================
    # Task 3: Validation for Job Satisfaction Variables
    # =====================================================================
    print("\n" + "="*60)
    print("Task 3: Validation for Job Satisfaction Variables")
    print("="*60)
    
    for col in job_sat_vars:
        if col in df.columns:
            # Before/after comparison requires original values; here we can only
            # check post-cleaning range, but we can still report current min/max.
            col_series = df[col]
            print(f"\n{col} (after cleaning):")
            print(f"  Min (ignoring NaN): {col_series.min()}")
            print(f"  Max (ignoring NaN): {col_series.max()}")
            out_of_range = ((col_series < 0) | (col_series > 10)).sum()
            print(f"  Out-of-range values remaining outside [0, 10]: {out_of_range}")
    
    # Ensure that max does not exceed 10 and min does not go below 0
    print("\n[CHECK] Post-cleaning logical range check for Job Satisfaction:")
    for col in job_sat_vars:
        if col in df.columns:
            col_series = df[col]
            valid_mask = col_series.notna()
            if valid_mask.any():
                min_val = col_series[valid_mask].min()
                max_val = col_series[valid_mask].max()
                print(f"  {col}: min={min_val}, max={max_val}")
            else:
                print(f"  {col}: all values are NaN after cleaning")
    
    # =====================================================================
    # Task 4: Update Global merged_df
    # =====================================================================
    print("\n" + "="*60)
    print("Task 4: Updating merged_df with Cleaned Data")
    print("="*60)
    
    merged_df = df
    print(f"[OK] merged_df updated. New shape: {merged_df.shape}")
else:
    print("\n[ERROR] 'merged_df' not found. Please run the merge and trimming steps first (Cells 2 and 3, and 4).")



Step 4.5 - Logical Range Validation and Cleaning

Original merged_df shape: (12939, 76)

Task 1: Defining Logical Range Rules
Job Satisfaction variables: ['cw23p133', 'cw24q133'] -> [0, 10]
Big Five items: cp24p020–cp24p069 (50 items) -> [1, 5]
Work characteristics (1–5 Likert): ['cw23p429', 'cw24q429', 'cw23p428', 'cw24q428', 'cw23p430', 'cw24q430'] -> [1, 5]
Health variables (Likert): ['ch23p022', 'ch24q022', 'ch23p004', 'ch24p004'] -> [1, 5]
WFC binary indicators: ['cw23p391', 'cw24q391'] -> [1, 2]
Remote work hours: ['cw23p610', 'cw24q610'] -> [0, 168]
Control variables: cw23p003 -> [16, 100], cf23p455 -> [0, 20]

Task 2: Applying Logical Range Cleaning
[CLEAN] cw23p133: 48 values outside [0, 10] -> set to NaN
[CLEAN] cw24q133: 58 values outside [0, 10] -> set to NaN
[CLEAN] cw23p391: 1224 values outside [1, 2] -> set to NaN
[CLEAN] cw24q391: 1350 values outside [1, 2] -> set to NaN

[OK] Logical cleaning applied to 4 columns
Total invalid values set to NaN: 2680

Task 3: Validatio

In [16]:
# Cell 6:  Big Five Personality Scales and Core Delta Variables Creation
# Big Five Personality Scales and Core Delta Variables Creation
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("Big Five Personality Scales & Core Delta Variables")
print("="*60)

if 'merged_df' in globals() and merged_df is not None:
    df = merged_df.copy()
    
    # ============================================================================
    # Task 1: Create Big Five Personality Scales (W16 Baseline)
    # ============================================================================
    print("\n" + "="*60)
    print("Task 1: Creating Big Five Personality Scales (W16)")
    print("="*60)
    
    # Define Big Five items and coding direction
    # Items are 1-5 scale, reverse coding uses: 6 - Value
    
    big_five_items = {
        'N': {  # Neuroticism (神经质)
            'forward': ['cp24p023', 'cp24p028', 'cp24p033', 'cp24p043', 'cp24p048', 
                       'cp24p053', 'cp24p058', 'cp24p063', 'cp24p068'],
            'reverse': ['cp24p038']
        },
        'E': {  # Extraversion (外向性)
            'forward': ['cp24p020', 'cp24p030', 'cp24p040', 'cp24p050', 'cp24p060'],
            'reverse': ['cp24p025', 'cp24p035', 'cp24p045', 'cp24p055', 'cp24p065']
        },
        'C': {  # Conscientiousness (尽责性)
            'forward': ['cp24p022', 'cp24p032', 'cp24p042', 'cp24p052', 'cp24p062', 'cp24p067'],
            'reverse': ['cp24p027', 'cp24p037', 'cp24p047', 'cp24p057']
        },
        'A': {  # Agreeableness (宜人性)
            'forward': ['cp24p026', 'cp24p036', 'cp24p046', 'cp24p061'],
            'reverse': ['cp24p021', 'cp24p031', 'cp24p041', 'cp24p051', 'cp24p056', 'cp24p066']
        },
        'O': {  # Openness (开放性)
            'forward': ['cp24p034', 'cp24p044', 'cp24p054', 'cp24p064', 'cp24p069'],
            'reverse': ['cp24p024', 'cp24p029', 'cp24p039', 'cp24p049', 'cp24p059']
        }
    }
    
    # Function to calculate Cronbach's Alpha
    def cronbach_alpha(df_items):
        """
        Calculate Cronbach's Alpha for a set of items.
        
        Parameters:
        -----------
        df_items : pandas.DataFrame
            DataFrame with items as columns
        
        Returns:
        --------
        float
            Cronbach's Alpha coefficient
        """
        # Remove rows with any missing values
        df_clean = df_items.dropna()
        
        if len(df_clean) < 2 or df_clean.shape[1] < 2:
            return np.nan
        
        # Calculate item variances and total variance
        item_variances = df_clean.var(axis=0, ddof=1)
        total_variance = df_clean.sum(axis=1).var(ddof=1)
        
        # Cronbach's Alpha formula
        n_items = df_clean.shape[1]
        if total_variance == 0:
            return np.nan
        
        alpha = (n_items / (n_items - 1)) * (1 - item_variances.sum() / total_variance)
        return alpha
    
    # Process each Big Five dimension
    big_five_scores = {}
    cronbach_alphas = {}
    
    for dimension, items in big_five_items.items():
        print(f"\nProcessing {dimension} ({'Neuroticism' if dimension == 'N' else 'Extraversion' if dimension == 'E' else 'Conscientiousness' if dimension == 'C' else 'Agreeableness' if dimension == 'A' else 'Openness'})...")
        
        # Collect all items for this dimension
        all_items = []
        processed_items = []
        
        # Process forward items
        for item in items['forward']:
            if item in df.columns:
                # Convert to numeric, coercing errors to NaN
                df[item] = pd.to_numeric(df[item], errors='coerce')
                all_items.append(item)
                processed_items.append(item)
            else:
                print(f"  [WARNING] Warning: {item} not found in dataset")
        
        # Process reverse items
        for item in items['reverse']:
            if item in df.columns:
                # Convert to numeric, coercing errors to NaN
                numeric_values = pd.to_numeric(df[item], errors='coerce')
                # Reverse code: 6 - Value (only for valid numeric values)
                reverse_col = f"{item}_rev"
                df[reverse_col] = 6 - numeric_values
                all_items.append(item)
                processed_items.append(reverse_col)
            else:
                print(f"  [WARNING] Warning: {item} not found in dataset")
        
        if len(processed_items) == 0:
            print(f"  [ERROR] Error: No items found for {dimension}")
            continue
        
        # Calculate mean score
        score_col = f"{dimension}_Score"
        df[score_col] = df[processed_items].mean(axis=1)
        big_five_scores[dimension] = score_col
        
        # Calculate Cronbach's Alpha
        df_for_alpha = df[processed_items].copy()
        alpha = cronbach_alpha(df_for_alpha)
        cronbach_alphas[dimension] = alpha
        
        print(f"  [OK] Created {score_col}")
        print(f"    Items used: {len(processed_items)} ({len(items['forward'])} forward, {len(items['reverse'])} reverse)")
        print(f"    Cronbach's Alpha: {alpha:.4f}" if not np.isnan(alpha) else "    Cronbach's Alpha: N/A (insufficient data)")
    
    # ============================================================================
    # Task 2: Calculate Core 5 Delta Change Variables
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Creating Core Delta Change Variables")
    print("="*60)
    
    delta_variables = {}
    
    # Delta_JobSatisfaction = cw24q133 - cw23p133
    if 'cw24q133' in df.columns and 'cw23p133' in df.columns:
        w24 = pd.to_numeric(df['cw24q133'], errors='coerce')
        w23 = pd.to_numeric(df['cw23p133'], errors='coerce')
        df['Delta_JobSatisfaction'] = w24 - w23
        delta_variables['Delta_JobSatisfaction'] = ['cw24q133', 'cw23p133']
        print("[OK] Created Delta_JobSatisfaction = cw24q133 - cw23p133")
    else:
        missing = [v for v in ['cw24q133', 'cw23p133'] if v not in df.columns]
        print(f"[WARNING] Warning: Missing variables for Delta_JobSatisfaction: {missing}")
    
    # Delta_Autonomy = (5 - cw24q429) - (5 - cw23p429)
    if 'cw24q429' in df.columns and 'cw23p429' in df.columns:
        w24 = pd.to_numeric(df['cw24q429'], errors='coerce')
        w23 = pd.to_numeric(df['cw23p429'], errors='coerce')
        df['Delta_Autonomy'] = (5 - w24) - (5 - w23)
        delta_variables['Delta_Autonomy'] = ['cw24q429', 'cw23p429']
        print("[OK] Created Delta_Autonomy = (5 - cw24q429) - (5 - cw23p429)")
    else:
        missing = [v for v in ['cw24q429', 'cw23p429'] if v not in df.columns]
        print(f"[WARNING] Warning: Missing variables for Delta_Autonomy: {missing}")
    
    # Delta_Workload = cw24q428 - cw23p428
    if 'cw24q428' in df.columns and 'cw23p428' in df.columns:
        w24 = pd.to_numeric(df['cw24q428'], errors='coerce')
        w23 = pd.to_numeric(df['cw23p428'], errors='coerce')
        df['Delta_Workload'] = w24 - w23
        delta_variables['Delta_Workload'] = ['cw24q428', 'cw23p428']
        print("[OK] Created Delta_Workload = cw24q428 - cw23p428")
    else:
        missing = [v for v in ['cw24q428', 'cw23p428'] if v not in df.columns]
        print(f"[WARNING] Warning: Missing variables for Delta_Workload: {missing}")
    
    # Delta_Skills = cw24q430 - cw23p430
    if 'cw24q430' in df.columns and 'cw23p430' in df.columns:
        w24 = pd.to_numeric(df['cw24q430'], errors='coerce')
        w23 = pd.to_numeric(df['cw23p430'], errors='coerce')
        df['Delta_Skills'] = w24 - w23
        delta_variables['Delta_Skills'] = ['cw24q430', 'cw23p430']
        print("[OK] Created Delta_Skills = cw24q430 - cw23p430")
    else:
        missing = [v for v in ['cw24q430', 'cw23p430'] if v not in df.columns]
        print(f"[WARNING] Warning: Missing variables for Delta_Skills: {missing}")
    
    # Delta_RemoteHours = cw24q610 - cw23p610
    if 'cw24q610' in df.columns and 'cw23p610' in df.columns:
        w24 = pd.to_numeric(df['cw24q610'], errors='coerce')
        w23 = pd.to_numeric(df['cw23p610'], errors='coerce')
        df['Delta_RemoteHours'] = w24 - w23
        delta_variables['Delta_RemoteHours'] = ['cw24q610', 'cw23p610']
        print("[OK] Created Delta_RemoteHours = cw24q610 - cw23p610")
    else:
        missing = [v for v in ['cw24q610', 'cw23p610'] if v not in df.columns]
        print(f"[WARNING] Warning: Missing variables for Delta_RemoteHours: {missing}")
    
    # ============================================================================
    # Task 3: Output Validation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: Validation Output")
    print("="*60)
    
    # Print Cronbach's Alpha values
    print("\n" + "-"*60)
    print("Cronbach's Alpha Coefficients for Big Five Scales")
    print("-"*60)
    print(f"\n{'Dimension':<15} {'Scale Name':<25} {'Alpha':<10} {'Interpretation'}")
    print("-" * 70)
    
    alpha_interpretation = {
        'Excellent': (0.9, 1.0),
        'Good': (0.8, 0.9),
        'Acceptable': (0.7, 0.8),
        'Questionable': (0.6, 0.7),
        'Poor': (0.0, 0.6)
    }
    
    dimension_names = {
        'N': 'Neuroticism',
        'E': 'Extraversion',
        'C': 'Conscientiousness',
        'A': 'Agreeableness',
        'O': 'Openness'
    }
    
    for dim in ['N', 'E', 'C', 'A', 'O']:
        if dim in cronbach_alphas:
            alpha = cronbach_alphas[dim]
            if not np.isnan(alpha):
                for label, (low, high) in alpha_interpretation.items():
                    if low <= alpha < high:
                        interpretation = label
                        break
                else:
                    interpretation = "N/A"
            else:
                interpretation = "N/A"
            
            print(f"{dim:<15} {dimension_names[dim]:<25} {alpha:>8.4f}  {interpretation}")
        else:
            print(f"{dim:<15} {dimension_names[dim]:<25} {'N/A':>8}  Not calculated")
    
    # Print Big Five scores summary statistics
    print("\n" + "-"*60)
    print("Big Five Personality Scores - Summary Statistics")
    print("-"*60)
    
    big_five_cols = [f"{dim}_Score" for dim in ['N', 'E', 'C', 'A', 'O'] if f"{dim}_Score" in df.columns]
    
    if big_five_cols:
        print("\n" + df[big_five_cols].describe().to_string())
        
        # Additional statistics
        print("\n" + "-"*60)
        print("Additional Statistics")
        print("-"*60)
        print(f"\n{'Dimension':<15} {'Mean':<10} {'Std':<10} {'Min':<10} {'Max':<10} {'Missing':<10}")
        print("-" * 65)
        
        for col in big_five_cols:
            mean_val = df[col].mean()
            std_val = df[col].std()
            min_val = df[col].min()
            max_val = df[col].max()
            missing = df[col].isna().sum()
            print(f"{col:<15} {mean_val:>9.3f} {std_val:>9.3f} {min_val:>9.3f} {max_val:>9.3f} {missing:>9,}")
    else:
        print("\n[WARNING] No Big Five scores found in dataset")
    
    # Print Delta variables summary statistics
    print("\n" + "-"*60)
    print("Delta Change Variables - Summary Statistics")
    print("-"*60)
    
    delta_cols = [col for col in ['Delta_JobSatisfaction', 'Delta_Autonomy', 'Delta_Workload', 
                                   'Delta_Skills', 'Delta_RemoteHours'] if col in df.columns]
    
    if delta_cols:
        print("\n" + df[delta_cols].describe().to_string())
        
        # Additional statistics
        print("\n" + "-"*60)
        print("Additional Statistics")
        print("-"*60)
        print(f"\n{'Variable':<25} {'Mean':<12} {'Std':<12} {'Min':<12} {'Max':<12} {'Missing':<10}")
        print("-" * 85)
        
        for col in delta_cols:
            mean_val = df[col].mean()
            std_val = df[col].std()
            min_val = df[col].min()
            max_val = df[col].max()
            missing = df[col].isna().sum()
            print(f"{col:<25} {mean_val:>11.3f} {std_val:>11.3f} {min_val:>11.3f} {max_val:>11.3f} {missing:>9,}")
        
        # Show correlation between delta variables
        if len(delta_cols) > 1:
            print("\n" + "-"*60)
            print("Correlation Matrix of Delta Variables")
            print("-"*60)
            corr_matrix = df[delta_cols].corr()
            print("\n" + corr_matrix.to_string())
    else:
        print("\n[WARNING] No Delta variables found in dataset")
    
    # Update merged_df
    merged_df = df
    
    print("\n" + "="*60)
    print("[OK] Big Five and Delta variables creation completed!")
    print("Updated dataset is available as 'merged_df' variable")
    print("="*60)
    
else:
    print(" Error: 'merged_df' not found. Please run previous cells first.")


Big Five Personality Scales & Core Delta Variables

Task 1: Creating Big Five Personality Scales (W16)

Processing N (Neuroticism)...
  [OK] Created N_Score
    Items used: 10 (9 forward, 1 reverse)
    Cronbach's Alpha: 0.8205

Processing E (Extraversion)...
  [OK] Created E_Score
    Items used: 10 (5 forward, 5 reverse)
    Cronbach's Alpha: 0.8825

Processing C (Conscientiousness)...
  [OK] Created C_Score
    Items used: 10 (6 forward, 4 reverse)
    Cronbach's Alpha: 0.7903

Processing A (Agreeableness)...
  [OK] Created A_Score
    Items used: 10 (4 forward, 6 reverse)
    Cronbach's Alpha: 0.5629

Processing O (Openness)...
  [OK] Created O_Score
    Items used: 10 (5 forward, 5 reverse)
    Cronbach's Alpha: 0.4873

Task 2: Creating Core Delta Change Variables
[OK] Created Delta_JobSatisfaction = cw24q133 - cw23p133
[OK] Created Delta_Autonomy = (5 - cw24q429) - (5 - cw23p429)
[OK] Created Delta_Workload = cw24q428 - cw23p428
[OK] Created Delta_Skills = cw24q430 - cw23p430
[OK

In [22]:
# Cell 7:  Remaining Delta Variables & Baseline Control Variables Preparation
#  Remaining Delta Variables & Baseline Control Variables Preparation
print("="*60)
print("Module 2/4: Remaining Delta Variables & Baseline Controls")
print("="*60)

if 'merged_df' in globals() and merged_df is not None:
    df = merged_df.copy()
    
    # ============================================================================
    # Task 1: Calculate Last 2 Delta Variables (WLB/Health Stress)
    # ============================================================================
    print("\n" + "="*60)
    print("Task 1: Calculating Last 2 Delta Variables")
    print("="*60)
    
    # Delta_WFC_Proxy = cw24q391 - cw23p391
    if 'cw24q391' in df.columns and 'cw23p391' in df.columns:
        w24 = pd.to_numeric(df['cw24q391'], errors='coerce')
        w23 = pd.to_numeric(df['cw23p391'], errors='coerce')
        df['Delta_WFC_Proxy'] = w24 - w23
        print("[OK] Created Delta_WFC_Proxy = cw24q391 - cw23p391")
    else:
        missing = [v for v in ['cw24q391', 'cw23p391'] if v not in df.columns]
        print(f"[WARNING] Warning: Missing variables for Delta_WFC_Proxy: {missing}")
    
    # Delta_Health_Hindrance = ch24q022 - ch23p022
    if 'ch24q022' in df.columns and 'ch23p022' in df.columns:
        w24 = pd.to_numeric(df['ch24q022'], errors='coerce')
        w23 = pd.to_numeric(df['ch23p022'], errors='coerce')
        df['Delta_Health_Hindrance'] = w24 - w23
        print("[OK] Created Delta_Health_Hindrance = ch24q022 - ch23p022")
    else:
        missing = [v for v in ['ch24q022', 'ch23p022'] if v not in df.columns]
        print(f"[WARNING] Warning: Missing variables for Delta_Health_Hindrance: {missing}")
    
    # ============================================================================
    # Task 2: Baseline Continuous Variables Selection
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Baseline Continuous Variables Selection")
    print("="*60)
    
    baseline_continuous_vars = {
        'Age': 'cw23p003',
        'Children_Count': 'cf23p455',
        'SelfRated_Health': 'ch23p004'
    }
    
    baseline_vars_found = {}
    baseline_vars_missing = {}
    
    for var_name, var_code in baseline_continuous_vars.items():
        if var_code in df.columns:
            # Convert to numeric
            df[var_code] = pd.to_numeric(df[var_code], errors='coerce')
            baseline_vars_found[var_name] = var_code
            print(f"[OK] Found {var_name}: {var_code}")
        else:
            baseline_vars_missing[var_name] = var_code
            print(f"[WARNING] Warning: {var_name} ({var_code}) not found in dataset")
    
    # ============================================================================
    # Task 3: Education Level Recoding
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: Education Level Recoding")
    print("="*60)
    
    if 'cw23p005' in df.columns:
        # Convert to numeric
        df['cw23p005'] = pd.to_numeric(df['cw23p005'], errors='coerce')
        
        # Recode education levels
        # 1 (低教育): codes 1-7
        # 2 (中教育): codes 8-15
        # 3 (高教育): codes 16-22
        # 4 (最高教育): codes 23-28
        
        def recode_education(value):
            if pd.isna(value):
                return np.nan
            value = int(value)
            if 1 <= value <= 7:
                return 1  # 低教育
            elif 8 <= value <= 15:
                return 2  # 中教育
            elif 16 <= value <= 22:
                return 3  # 高教育
            elif 23 <= value <= 28:
                return 4  # 最高教育
            else:
                return np.nan  # Out of range
        
        df['Education_Recode'] = df['cw23p005'].apply(recode_education)
        
        # Count original values
        original_counts = df['cw23p005'].value_counts().sort_index()
        recoded_counts = df['Education_Recode'].value_counts().sort_index()
        
        print("[OK] Created Education_Recode from cw23p005")
        print(f"\n  Original education codes distribution:")
        print(f"    Range: {df['cw23p005'].min():.0f} - {df['cw23p005'].max():.0f}")
        print(f"    Unique values: {df['cw23p005'].nunique()}")
        print(f"\n  Recoded education levels:")
        print(f"    1 (低教育): {recoded_counts.get(1, 0):,} cases")
        print(f"    2 (中教育): {recoded_counts.get(2, 0):,} cases")
        print(f"    3 (高教育): {recoded_counts.get(3, 0):,} cases")
        print(f"    4 (最高教育): {recoded_counts.get(4, 0):,} cases")
        print(f"    Missing: {df['Education_Recode'].isna().sum():,} cases")
    else:
        print("[WARNING] Warning: cw23p005 (W16 highest education) not found in dataset")
    
    # ============================================================================
    # Task 4: Income Quartile Binning
    # ============================================================================
    print("\n" + "="*60)
    print("Task 4: Income Quartile Binning")
    print("="*60)
    
    if 'ci23p337' in df.columns:
        # Convert to numeric
        df['ci23p337'] = pd.to_numeric(df['ci23p337'], errors='coerce')
        
        # Calculate quartiles (excluding NaN values)
        income_valid = df['ci23p337'].dropna()
        
        if len(income_valid) > 0:
            # Calculate quartile boundaries
            q1 = income_valid.quantile(0.25)
            q2 = income_valid.quantile(0.50)  # median
            q3 = income_valid.quantile(0.75)
            
            print(f"  Income quartile boundaries:")
            print(f"    Q1 (25th percentile): {q1:,.2f}")
            print(f"    Q2 (50th percentile/median): {q2:,.2f}")
            print(f"    Q3 (75th percentile): {q3:,.2f}")
            
            # Create quartile bins
            def assign_quartile(value):
                if pd.isna(value):
                    return np.nan
                if value <= q1:
                    return 1  # Q1 (lowest)
                elif value <= q2:
                    return 2  # Q2
                elif value <= q3:
                    return 3  # Q3
                else:
                    return 4  # Q4 (highest)
            
            df['Income_Quartile'] = df['ci23p337'].apply(assign_quartile)
            
            quartile_counts = df['Income_Quartile'].value_counts().sort_index()
            
            print("[OK] Created Income_Quartile from ci23p337")
            print(f"\n  Income quartile distribution:")
            for q in [1, 2, 3, 4]:
                count = quartile_counts.get(q, 0)
                pct = (count / len(income_valid)) * 100 if len(income_valid) > 0 else 0
                print(f"    Quartile {q}: {count:,} cases ({pct:.1f}%)")
            print(f"    Missing: {df['Income_Quartile'].isna().sum():,} cases")
        else:
            print("[WARNING] Warning: No valid income values found in ci23p337")
    else:
        print("[WARNING] Warning: ci23p337 (W16 total taxable income) not found in dataset")
    
    # ============================================================================
    # Task 5: Output Validation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 5: Validation Output")
    print("="*60)
    
    # Print Delta variables summary statistics
    print("\n" + "-"*60)
    print("Delta Variables - Summary Statistics")
    print("-"*60)
    
    delta_cols = [col for col in ['Delta_WFC_Proxy', 'Delta_Health_Hindrance'] 
                  if col in df.columns]
    
    if delta_cols:
        print("\n" + df[delta_cols].describe().to_string())
        
        # Additional statistics
        print("\n" + "-"*60)
        print("Additional Statistics")
        print("-"*60)
        print(f"\n{'Variable':<25} {'Mean':<12} {'Std':<12} {'Min':<12} {'Max':<12} {'Missing':<10}")
        print("-" * 85)
        
        for col in delta_cols:
            mean_val = df[col].mean()
            std_val = df[col].std()
            min_val = df[col].min()
            max_val = df[col].max()
            missing = df[col].isna().sum()
            print(f"{col:<25} {mean_val:>11.3f} {std_val:>11.3f} {min_val:>11.3f} {max_val:>11.3f} {missing:>9,}")
    else:
        print("\n[WARNING] No Delta variables found in dataset")
    
    # Print Education_Recode value counts
    print("\n" + "-"*60)
    print("Education_Recode - Value Counts")
    print("-"*60)
    
    if 'Education_Recode' in df.columns:
        edu_counts = df['Education_Recode'].value_counts().sort_index()
        edu_pct = df['Education_Recode'].value_counts(normalize=True).sort_index() * 100
        
        print(f"\n{'Level':<10} {'Label':<15} {'Count':<15} {'Percentage':<10}")
        print("-" * 50)
        
        edu_labels = {
            1: '低教育 (Low)',
            2: '中教育 (Medium)',
            3: '高教育 (High)',
            4: '最高教育 (Highest)'
        }
        
        for level in [1, 2, 3, 4]:
            count = edu_counts.get(level, 0)
            pct = edu_pct.get(level, 0)
            label = edu_labels.get(level, 'Unknown')
            print(f"{level:<10} {label:<15} {count:<15,} {pct:>8.2f}%")
        
        missing_edu = df['Education_Recode'].isna().sum()
        if missing_edu > 0:
            missing_pct = (missing_edu / len(df)) * 100
            print(f"{'Missing':<10} {'':<15} {missing_edu:<15,} {missing_pct:>8.2f}%")
        
        print(f"\nTotal cases: {len(df):,}")
    else:
        print("\n[WARNING] Education_Recode not found in dataset")
    
    # Print Income_Quartile value counts
    print("\n" + "-"*60)
    print("Income_Quartile - Value Counts")
    print("-"*60)
    
    if 'Income_Quartile' in df.columns:
        income_counts = df['Income_Quartile'].value_counts().sort_index()
        income_pct = df['Income_Quartile'].value_counts(normalize=True).sort_index() * 100
        
        print(f"\n{'Quartile':<10} {'Label':<20} {'Count':<15} {'Percentage':<10}")
        print("-" * 55)
        
        quartile_labels = {
            1: 'Q1 (Lowest 25%)',
            2: 'Q2 (25-50%)',
            3: 'Q3 (50-75%)',
            4: 'Q4 (Highest 25%)'
        }
        
        for q in [1, 2, 3, 4]:
            count = income_counts.get(q, 0)
            pct = income_pct.get(q, 0)
            label = quartile_labels.get(q, 'Unknown')
            print(f"{q:<10} {label:<20} {count:<15,} {pct:>8.2f}%")
        
        missing_income = df['Income_Quartile'].isna().sum()
        if missing_income > 0:
            missing_pct = (missing_income / len(df)) * 100
            print(f"{'Missing':<10} {'':<20} {missing_income:<15,} {missing_pct:>8.2f}%")
        
        print(f"\nTotal cases: {len(df):,}")
    else:
        print("\n[WARNING] Income_Quartile not found in dataset")
    
    # Print baseline continuous variables summary
    if baseline_vars_found:
        print("\n" + "-"*60)
        print("Baseline Continuous Variables - Summary Statistics")
        print("-"*60)
        
        baseline_cols = list(baseline_vars_found.values())
        print("\n" + df[baseline_cols].describe().to_string())
    
    # Update merged_df
    merged_df = df
    
    print("\n" + "="*60)
    print("[OK] Module 2/4 completed!")
    print("Updated dataset is available as 'merged_df' variable")
    print("="*60)
    
else:
    print("\n[ERROR] Error: 'merged_df' not found. Please run previous cells first.")


Module 2/4: Remaining Delta Variables & Baseline Controls

Task 1: Calculating Last 2 Delta Variables
[OK] Created Delta_WFC_Proxy = cw24q391 - cw23p391
[OK] Created Delta_Health_Hindrance = ch24q022 - ch23p022

Task 2: Baseline Continuous Variables Selection
[OK] Found Age: cw23p003
[OK] Found Children_Count: cf23p455
[OK] Found SelfRated_Health: ch23p004

Task 3: Education Level Recoding
[OK] Created Education_Recode from cw23p005

  Original education codes distribution:
    Range: 1 - 28
    Unique values: 28

  Recoded education levels:
    1 (低教育): 625 cases
    2 (中教育): 1,265 cases
    3 (高教育): 2,349 cases
    4 (最高教育): 900 cases
    Missing: 7,800 cases

Task 4: Income Quartile Binning
  Income quartile boundaries:
    Q1 (25th percentile): -9,999,999,999.00
    Q2 (50th percentile/median): -9,999,999,998.00
    Q3 (75th percentile): 32,142.50
[OK] Created Income_Quartile from ci23p337

  Income quartile distribution:
    Quartile 1: 1,998 cases (41.4%)
    Quartile 2: 682 case

In [23]:
# Cell 8:  MICE Imputation using IterativeImputer
print("="*60)
print("MICE Imputation using IterativeImputer")
print("="*60)

if 'merged_df' in globals() and merged_df is not None:
    df = merged_df.copy()
    
    print(f"\nInput dataset shape: {df.shape}")
    print(f"  Rows: {df.shape[0]:,}")
    print(f"  Columns: {df.shape[1]:,}")
    
    # ============================================================================
    # Task 1: Define Imputation Feature Set
    # ============================================================================
    print("\n" + "="*60)
    print("Task 1: Defining Imputation Feature Set")
    print("="*60)
    
    imputation_features = []
    
    # 1. Big Five Scores (5 features)
    big_five_scores = ['N_Score', 'E_Score', 'C_Score', 'A_Score', 'O_Score']
    for score in big_five_scores:
        if score in df.columns:
            imputation_features.append(score)
    print(f"\n1. Big Five Scores: {len([s for s in big_five_scores if s in df.columns])} features")
    for score in big_five_scores:
        if score in df.columns:
            print(f"   - {score}")
    
    # 2. Delta change variables (7 features)
    delta_variables = [
        'Delta_JobSatisfaction',
        'Delta_Autonomy',
        'Delta_Workload',
        'Delta_Skills',
        'Delta_RemoteHours',
        'Delta_WFC_Proxy',
        'Delta_Health_Hindrance'
    ]
    for delta_var in delta_variables:
        if delta_var in df.columns:
            imputation_features.append(delta_var)
    print(f"\n2. Delta Change Variables: {len([v for v in delta_variables if v in df.columns])} features")
    for delta_var in delta_variables:
        if delta_var in df.columns:
            print(f"   - {delta_var}")
    
    # 3. Continuous control variables (3 features)
    continuous_controls = [
        'cw23p003',      # Age
        'cf23p455',      # Children Count
        'ch23p004'       # Self-rated Health
    ]
    for control_var in continuous_controls:
        if control_var in df.columns:
            imputation_features.append(control_var)
    print(f"\n3. Continuous Control Variables: {len([v for v in continuous_controls if v in df.columns])} features")
    for control_var in continuous_controls:
        if control_var in df.columns:
            print(f"   - {control_var}")
    
    # 4. Income and occupation variables (for auxiliary imputation)
    income_occupation_vars = [
        'ci23p337',      # W16 Income
        'ci24q337',      # W17 Income
        'ci23p383',      # W16 Occupation
        'cw23p525',      # W16 Main Occupation
        'ci24q383',      # W17 Occupation
        'cw24q525'       # W17 Main Occupation
    ]
    for var in income_occupation_vars:
        if var in df.columns:
            imputation_features.append(var)
    print(f"\n4. Income and Occupation Variables (auxiliary): {len([v for v in income_occupation_vars if v in df.columns])} features")
    for var in income_occupation_vars:
        if var in df.columns:
            print(f"   - {var}")
    
    # Remove duplicates while preserving order
    imputation_features = list(dict.fromkeys(imputation_features))
    
    # Exclude OHE dummy variables and ID
    # Identify dummy variables (typically contain underscores and numbers, or are binary 0/1)
    excluded_vars = ['nomem_encr']  # Always exclude ID
    
    # Exclude OHE dummy variables (they typically have patterns like 'var_name_2.0', 'var_name_3.0', etc.)
    dummy_patterns = []
    for col in df.columns:
        # Check if column looks like a dummy variable (contains underscore and ends with number/float pattern)
        if '_' in col and col not in imputation_features:
            # Check if it's likely a dummy variable (binary or categorical encoded)
            if df[col].dtype in ['int64', 'float64']:
                unique_vals = df[col].dropna().unique()
                if len(unique_vals) <= 2 and set(unique_vals).issubset({0, 1, 0.0, 1.0}):
                    dummy_patterns.append(col)
    
    excluded_vars.extend(dummy_patterns)
    
    # Remove excluded variables from imputation features
    imputation_features = [f for f in imputation_features if f not in excluded_vars]
    
    print(f"\n" + "-"*60)
    print(f"Imputation Feature Set Summary:")
    print(f"-"*60)
    print(f"  Total features for imputation: {len(imputation_features)}")
    print(f"  Excluded variables: {len(excluded_vars)} (ID + OHE dummies)")
    
    # Check which features exist in dataset
    available_imputation_features = [f for f in imputation_features if f in df.columns]
    missing_imputation_features = [f for f in imputation_features if f not in df.columns]
    
    if missing_imputation_features:
        print(f"\n  [WARNING] Missing features: {len(missing_imputation_features)}")
        print(f"    {', '.join(missing_imputation_features[:10])}{'...' if len(missing_imputation_features) > 10 else ''}")
    
    print(f"\n  Available features for imputation: {len(available_imputation_features)}")
    
    # Check missing values in imputation features
    imputation_data = df[available_imputation_features].copy()
    missing_counts = imputation_data.isna().sum()
    total_missing = missing_counts.sum()
    
    print(f"\n  Missing values in imputation features: {total_missing:,}")
    if total_missing > 0:
        print(f"  Features with missing values:")
        missing_features = missing_counts[missing_counts > 0].sort_values(ascending=False)
        for feat, count in missing_features.head(10).items():
            pct = (count / len(imputation_data)) * 100
            print(f"    - {feat}: {count:,} ({pct:.1f}%)")
        if len(missing_features) > 10:
            print(f"    ... and {len(missing_features) - 10} more features with missing values")
    
    # ============================================================================
    # Task 2: Execute MICE Imputation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Executing MICE Imputation (M=5)")
    print("="*60)
    
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer
    from sklearn.linear_model import BayesianRidge
    import numpy as np
    
    # Prepare data for imputation
    X_impute = imputation_data.copy()
    
    # Convert all columns to numeric, coercing errors to NaN
    print(f"\nConverting features to numeric type...")
    numeric_columns = []
    non_numeric_columns = []
    
    for col in available_imputation_features:
        if col in X_impute.columns:
            # Try to convert to numeric
            original_dtype = X_impute[col].dtype
            X_impute[col] = pd.to_numeric(X_impute[col], errors='coerce')
            
            # Check if conversion was successful
            if X_impute[col].dtype in ['int64', 'float64']:
                numeric_columns.append(col)
            else:
                non_numeric_columns.append(col)
                print(f"  [WARNING] {col}: Could not convert to numeric (original dtype: {original_dtype})")
    
    # Remove non-numeric columns from imputation
    if non_numeric_columns:
        print(f"\n  Excluding {len(non_numeric_columns)} non-numeric columns from imputation:")
        for col in non_numeric_columns:
            print(f"    - {col}")
        available_imputation_features = [f for f in available_imputation_features if f in numeric_columns]
        X_impute = X_impute[numeric_columns].copy()
    
    # Ensure all remaining columns are float64 for IterativeImputer
    for col in X_impute.columns:
        if X_impute[col].dtype == 'int64':
            X_impute[col] = X_impute[col].astype('float64')
    
    print(f"\nImputation setup:")
    print(f"  Features to impute: {len(available_imputation_features)}")
    print(f"  Observations: {len(X_impute):,}")
    print(f"  Total missing values: {X_impute.isna().sum().sum():,}")
    
    # Final check: ensure no string values remain
    for col in X_impute.columns:
        if X_impute[col].dtype == 'object':
            print(f"  [ERROR] {col} is still object type - will be excluded")
            available_imputation_features = [f for f in available_imputation_features if f != col]
    
    if len(available_imputation_features) == 0:
        raise ValueError("No numeric features available for imputation!")
    
    X_impute = X_impute[available_imputation_features].copy()
    
    # Final validation: ensure X_impute is purely numeric
    # Replace any remaining non-numeric values with NaN
    for col in X_impute.columns:
        # Convert to numeric again to catch any remaining issues
        X_impute[col] = pd.to_numeric(X_impute[col], errors='coerce')
        # Ensure float64
        if X_impute[col].dtype != 'float64':
            X_impute[col] = X_impute[col].astype('float64')
    
    # Verify no object columns remain
    object_cols = X_impute.select_dtypes(include=['object']).columns
    if len(object_cols) > 0:
        print(f"\n  [WARNING] Removing {len(object_cols)} object columns: {list(object_cols)}")
        X_impute = X_impute.select_dtypes(exclude=['object'])
        available_imputation_features = [f for f in available_imputation_features if f in X_impute.columns]
    
    # Convert to numpy array to ensure compatibility
    X_impute_values = X_impute.values.astype('float64')
    
    print(f"\n  Final validation:")
    print(f"    Data type: {type(X_impute_values)}")
    print(f"    Shape: {X_impute_values.shape}")
    print(f"    Dtype: {X_impute_values.dtype}")
    print(f"    Contains NaN: {np.isnan(X_impute_values).any()}")
    
    # Initialize IterativeImputer with BayesianRidge as estimator
    imputer = IterativeImputer(
        estimator=BayesianRidge(),
        max_iter=10,
        random_state=42,
        imputation_order='ascending',
        n_nearest_features=None,
        verbose=0
    )
    
    print(f"\nIterativeImputer configuration:")
    print(f"  Estimator: BayesianRidge")
    print(f"  Max iterations: 10")
    print(f"  Random state: 42")
    
    # Perform 5 imputations (M=5)
    M = 5
    imputed_dataframes = []
    
    print(f"\nPerforming {M} imputations...")
    print("-"*60)
    
    for m in range(M):
        print(f"\nImputation {m+1}/{M}...", end=' ')
        
        # Fit and transform (each call uses different random initialization)
        # Set random_state differently for each imputation to get variation
        imputer_m = IterativeImputer(
            estimator=BayesianRidge(),
            max_iter=10,
            random_state=42 + m,  # Different seed for each imputation
            imputation_order='ascending',
            n_nearest_features=None,
            verbose=0
        )
        
        # Perform imputation (use numpy array to avoid pandas dtype issues)
        X_imputed_array = imputer_m.fit_transform(X_impute_values)
        
        # Convert back to DataFrame
        X_imputed_df = pd.DataFrame(
            X_imputed_array,
            columns=available_imputation_features,
            index=X_impute.index
        )
        
        # Create complete dataset by replacing imputed columns in original dataframe
        df_imputed = df.copy()
        for col in available_imputation_features:
            df_imputed[col] = X_imputed_df[col]
        
        imputed_dataframes.append(df_imputed)
        
        # Check missing values after imputation
        missing_after = df_imputed[available_imputation_features].isna().sum().sum()
        print(f"Complete (missing values: {missing_after})")
    
    print(f"\n" + "-"*60)
    print(f"[OK] Successfully created {M} imputed datasets")
    
    # ============================================================================
    # Task 3: Final Output and Validation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: Final Output and Validation")
    print("="*60)
    
    print(f"\n[OK] Successfully created {len(imputed_dataframes)} imputed datasets")
    
    # Validate first imputed dataset
    df_first = imputed_dataframes[0]
    
    print(f"\nValidation of first imputed dataset (imputed_dataframes[0]):")
    print("-"*60)
    
    # Check Delta_JobSatisfaction missing values
    if 'Delta_JobSatisfaction' in df_first.columns:
        missing_delta = df_first['Delta_JobSatisfaction'].isna().sum()
        print(f"  Delta_JobSatisfaction missing values: {missing_delta} (expected: 0)")
        if missing_delta == 0:
            print(f"    [OK] Delta_JobSatisfaction is complete")
        else:
            print(f"    [WARNING] Delta_JobSatisfaction still has {missing_delta} missing values")
    else:
        print(f"  [WARNING] Delta_JobSatisfaction not found in dataset")
    
    # Check Age (cw23p003) statistics
    if 'cw23p003' in df_first.columns:
        age_data = df_first['cw23p003'].dropna()
        if len(age_data) > 0:
            age_mean = age_data.mean()
            age_std = age_data.std()
            age_missing = df_first['cw23p003'].isna().sum()
            print(f"\n  Age (cw23p003) statistics:")
            print(f"    Mean: {age_mean:.2f}")
            print(f"    Std: {age_std:.2f}")
            print(f"    Missing values: {age_missing} (expected: 0)")
            if age_missing == 0:
                print(f"    [OK] Age is complete")
        else:
            print(f"  [WARNING] Age (cw23p003) has no valid values")
    else:
        print(f"  [WARNING] Age (cw23p003) not found in dataset")
    
    # Overall missing values check
    total_missing_first = df_first[available_imputation_features].isna().sum().sum()
    print(f"\n  Overall missing values in imputation features: {total_missing_first} (expected: 0)")
    if total_missing_first == 0:
        print(f"    [OK] All imputation features are complete")
    else:
        print(f"    [WARNING] Still have {total_missing_first} missing values")
    
    # Store imputed datasets in global variable
    globals()['imputed_dataframes'] = imputed_dataframes
    
    print(f"\n" + "="*60)
    print(f"[OK] MICE imputation completed!")
    print(f"  Created {M} imputed datasets")
    print(f"  Available as 'imputed_dataframes' variable (list of {M} DataFrames)")
    print("="*60)
    
else:
    print("\n[ERROR] Error: 'merged_df' not found. Please run previous cells first.")


MICE Imputation using IterativeImputer

Input dataset shape: (12939, 111)
  Rows: 12,939
  Columns: 111

Task 1: Defining Imputation Feature Set

1. Big Five Scores: 5 features
   - N_Score
   - E_Score
   - C_Score
   - A_Score
   - O_Score

2. Delta Change Variables: 7 features
   - Delta_JobSatisfaction
   - Delta_Autonomy
   - Delta_Workload
   - Delta_Skills
   - Delta_RemoteHours
   - Delta_WFC_Proxy
   - Delta_Health_Hindrance

3. Continuous Control Variables: 3 features
   - cw23p003
   - cf23p455
   - ch23p004

4. Income and Occupation Variables (auxiliary): 6 features
   - ci23p337
   - ci24q337
   - ci23p383
   - cw23p525
   - ci24q383
   - cw24q525

------------------------------------------------------------
Imputation Feature Set Summary:
------------------------------------------------------------
  Total features for imputation: 21
  Excluded variables: 1 (ID + OHE dummies)

  Available features for imputation: 21

  Missing values in imputation features: 182,507
  Feat

In [28]:
# Cell 9:  Module 3/4 & 4/4 - MICE Results Final Processing
# Module 3/4 & 4/4: MICE Results Final Processing
print("="*60)
print("Module 3/4 & 4/4: MICE Results Final Processing")
print("="*60)

if 'imputed_dataframes' in globals() and imputed_dataframes is not None and len(imputed_dataframes) > 0:
    print(f"\nFound {len(imputed_dataframes)} MICE imputed datasets")
    
    # ============================================================================
    # Loop through MICE datasets
    # ============================================================================
    M = len(imputed_dataframes)
    final_X_matrices = []
    final_Y_matrices = []
    
    for m_idx, df_imputed in enumerate(imputed_dataframes, 1):
        print(f"\n{'='*60}")
        print(f"Processing MICE Dataset {m_idx}/{M}")
        print(f"{'='*60}")
        
        df = df_imputed.copy()
        print(f"  Dataset shape: {df.shape}")
        
        # ============================================================================
        # Task 2: Categorical Encoding (OHE) and Binning
        # ============================================================================
        print(f"\n  Task 2: Categorical Encoding and Binning")
        print(f"  {'-'*60}")
        
        # 2.1 Education Recode (if not already done)
        if 'Education_Recode' not in df.columns and 'cw23p005' in df.columns:
            print(f"\n  Creating Education_Recode from cw23p005...")
            
            def recode_education(value):
                if pd.isna(value):
                    return np.nan
                value = float(value)
                if 1 <= value <= 7:
                    return 1  # Low education
                elif 8 <= value <= 15:
                    return 2  # Medium education
                elif 16 <= value <= 22:
                    return 3  # High education
                elif 23 <= value <= 28:
                    return 4  # Highest education
                else:
                    return np.nan
            
            df['cw23p005'] = pd.to_numeric(df['cw23p005'], errors='coerce')
            df['Education_Recode'] = df['cw23p005'].apply(recode_education)
            print(f"    [OK] Created Education_Recode")
        elif 'Education_Recode' in df.columns:
            print(f"  [OK] Education_Recode already exists")
        else:
            print(f"  [WARNING] cw23p005 not found, skipping Education_Recode")
        
        # 2.2 Income Quartile (if not already done)
        if 'Income_Quartile' not in df.columns and 'ci23p337' in df.columns:
            print(f"\n  Creating Income_Quartile from ci23p337...")
            
            income_data = pd.to_numeric(df['ci23p337'], errors='coerce')
            valid_income = income_data.dropna()
            
            if len(valid_income) > 0:
                quartiles = valid_income.quantile([0.25, 0.5, 0.75])
                
                def assign_quartile(value):
                    if pd.isna(value):
                        return np.nan
                    value = float(value)
                    if value <= quartiles[0.25]:
                        return 1
                    elif value <= quartiles[0.5]:
                        return 2
                    elif value <= quartiles[0.75]:
                        return 3
                    else:
                        return 4
                
                df['Income_Quartile'] = income_data.apply(assign_quartile)
                print(f"    [OK] Created Income_Quartile")
            else:
                print(f"    [WARNING] No valid income values found")
        elif 'Income_Quartile' in df.columns:
            print(f"  [OK] Income_Quartile already exists")
        else:
            print(f"  [WARNING] ci23p337 not found, skipping Income_Quartile")
        
        # 2.3 One-Hot Encoding (OHE)
        print(f"\n  Performing One-Hot Encoding...")
        
        ohe_variables = {
            'ch23p001': 'Gender',
            'Income_Quartile': 'Income_Quartile',
            'Education_Recode': 'Education_Recode'
        }
        
        ohe_dummy_vars = {}
        
        for var_name, var_label in ohe_variables.items():
            if var_name in df.columns:
                # Convert to numeric if needed
                df[var_name] = pd.to_numeric(df[var_name], errors='coerce')
                
                # Perform OHE
                dummies = pd.get_dummies(df[var_name], prefix=var_name, drop_first=True)
                
                # Add dummy variables to dataframe
                for dummy_col in dummies.columns:
                    df[dummy_col] = dummies[dummy_col]
                    if var_name not in ohe_dummy_vars:
                        ohe_dummy_vars[var_name] = []
                    ohe_dummy_vars[var_name].append(dummy_col)
                
                # Drop original variable
                df = df.drop(columns=[var_name])
                
                print(f"    [OK] {var_label}: Created {len(dummies.columns)} dummy variables, dropped original")
            else:
                print(f"    [WARNING] {var_label} ({var_name}) not found")
        
        # ============================================================================
        # Task 3: Build and Standardize Matrices
        # ============================================================================
        print(f"\n  Task 3: Building and Standardizing X and Y Matrices")
        print(f"  {'-'*60}")
        
        # Define Y (target variable)
        target_var = 'Delta_JobSatisfaction'
        if target_var in df.columns:
            Y = df[[target_var]].copy()
            print(f"  [OK] Target variable (Y): {target_var}")
            print(f"    Shape: {Y.shape}")
        else:
            print(f"  [ERROR] Target variable {target_var} not found!")
            continue
        
        # Define X features
        X_features = []
        
        # Big Five Scores (5 features)
        big_five_scores = ['N_Score', 'E_Score', 'C_Score', 'A_Score', 'O_Score']
        X_big_five = [f for f in big_five_scores if f in df.columns]
        X_features.extend(X_big_five)
        print(f"  [OK] Big Five Scores: {len(X_big_five)} features")
        
        # Delta variables (7 features)
        delta_vars = [
            'Delta_JobSatisfaction',  # Exclude from X (it's Y)
            'Delta_Autonomy',
            'Delta_Workload',
            'Delta_Skills',
            'Delta_RemoteHours',
            'Delta_WFC_Proxy',
            'Delta_Health_Hindrance'
        ]
        X_delta = [f for f in delta_vars if f in df.columns and f != target_var]
        X_features.extend(X_delta)
        print(f"  [OK] Delta Variables: {len(X_delta)} features")
        
        # Continuous control variables (3 features)
        continuous_controls = ['cw23p003', 'cf23p455', 'ch23p004']
        X_controls = [f for f in continuous_controls if f in df.columns]
        X_features.extend(X_controls)
        print(f"  [OK] Baseline Controls: {len(X_controls)} features")
        
        # OHE dummy variables
        X_dummy = []
        for var_name, dummy_cols in ohe_dummy_vars.items():
            for dummy_col in dummy_cols:
                if dummy_col in df.columns:
                    X_dummy.append(dummy_col)
        X_features.extend(X_dummy)
        print(f"  [OK] Dummy variables from OHE: {len(X_dummy)} features")
        
        # Create X matrix
        X = df[X_features].copy()
        print(f"\n  X matrix shape: {X.shape}")
        
        # Identify continuous features for standardization
        continuous_features = X_big_five + X_delta + X_controls
        
        # Standardize continuous features (Z-Score)
        print(f"\n  Standardizing {len(continuous_features)} continuous features...")
        from sklearn.preprocessing import StandardScaler
        
        scaler = StandardScaler()
        X_continuous_data = X[continuous_features].values
        X_continuous_scaled = scaler.fit_transform(X_continuous_data)
        
        # Replace standardized values in X
        for idx, feat in enumerate(continuous_features):
            X[feat] = X_continuous_scaled[:, idx]
        
        print(f"    [OK] Standardized {len(continuous_features)} continuous features")
        
        # Store matrices
        final_X_matrices.append(X)
        final_Y_matrices.append(Y)
        
        # ============================================================================
        # Task 4: Save to Disk
        # ============================================================================
        print(f"\n  Task 4: Saving to Disk")
        print(f"  {'-'*60}")
        
        X_filename = f'LISS_Final_X_M{m_idx}.csv'
        Y_filename = f'LISS_Final_Y_M{m_idx}.csv'
        
        X.to_csv(X_filename, index=True, encoding='utf-8')
        Y.to_csv(Y_filename, index=True, encoding='utf-8')
        
        import os
        X_size = os.path.getsize(X_filename) / (1024 * 1024)
        Y_size = os.path.getsize(Y_filename) / (1024 * 1024)
        
        print(f"    [OK] Saved X matrix: {X_filename} ({X_size:.2f} MB)")
        print(f"    [OK] Saved Y matrix: {Y_filename} ({Y_size:.2f} MB)")
    
    # ============================================================================
    # Task 5: Output Validation
    # ============================================================================
    print(f"\n{'='*60}")
    print("Task 5: Final Output Validation")
    print(f"{'='*60}")
    
    print(f"\n[OK] Successfully processed {M} MICE datasets")
    
    # Validate first dataset
    if len(final_X_matrices) > 0:
        X_first = final_X_matrices[0]
        Y_first = final_Y_matrices[0]
        
        print(f"\nValidation of first dataset (M1):")
        print(f"  X matrix shape: {X_first.shape}")
        print(f"  Y matrix shape: {Y_first.shape}")
        
        print(f"\n  X matrix column names ({len(X_first.columns)} features):")
        print(f"  {'-'*60}")
        for i, col in enumerate(X_first.columns, 1):
            print(f"    {i:2d}. {col}")
        
        print(f"\n  Y matrix:")
        print(f"    Target variable: {Y_first.columns[0]}")
        print(f"    Missing values: {Y_first.isna().sum().sum()}")
    
    # Store in global variables
    globals()['final_X_matrices'] = final_X_matrices
    globals()['final_Y_matrices'] = final_Y_matrices
    
    print(f"\n{'='*60}")
    print(f"[OK] Module 3/4 & 4/4 completed!")
    print(f"  Processed {M} MICE datasets")
    print(f"  Final X and Y matrices available as 'final_X_matrices' and 'final_Y_matrices'")
    print(f"  Saved files: LISS_Final_X_M1.csv to LISS_Final_X_M{M}.csv")
    print(f"               LISS_Final_Y_M1.csv to LISS_Final_Y_M{M}.csv")
    print("="*60)
    
else:
    print("\n[ERROR] Error: 'imputed_dataframes' not found. Please run Cell 7 (MICE imputation) first.")


Module 3/4 & 4/4: MICE Results Final Processing

Found 5 MICE imputed datasets

Processing MICE Dataset 1/5
  Dataset shape: (12939, 111)

  Task 2: Categorical Encoding and Binning
  ------------------------------------------------------------
  [OK] Education_Recode already exists
  [OK] Income_Quartile already exists

  Performing One-Hot Encoding...
    [OK] Gender: Created 2 dummy variables, dropped original
    [OK] Income_Quartile: Created 3 dummy variables, dropped original
    [OK] Education_Recode: Created 3 dummy variables, dropped original

  Task 3: Building and Standardizing X and Y Matrices
  ------------------------------------------------------------
  [OK] Target variable (Y): Delta_JobSatisfaction
    Shape: (12939, 1)
  [OK] Big Five Scores: 5 features
  [OK] Delta Variables: 6 features
  [OK] Baseline Controls: 3 features
  [OK] Dummy variables from OHE: 8 features

  X matrix shape: (12939, 22)

  Standardizing 14 continuous features...
    [OK] Standardized 14 co

In [29]:
# Cell 10:  Module 5/5 - Model Training & Randomized Search Benchmarking (Pooling)
# Module 5/5: Model Training & Randomized Search Benchmarking with MICE Pooling
import os
import sys
import subprocess
import shutil
import glob

# Fix OpenMP library dependencies for XGBoost and LightGBM (macOS)
# Find libomp.dylib in common locations
omp_paths = [
    '/Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/lib',
    '/opt/homebrew/opt/libomp/lib',
    '/usr/local/opt/libomp/lib',
]

omp_lib_path = None
for path in omp_paths:
    omp_lib = os.path.join(path, 'libomp.dylib')
    if os.path.exists(omp_lib):
        omp_lib_path = omp_lib
        break

# Also try glob pattern for R framework
if omp_lib_path is None:
    r_paths = glob.glob('/Library/Frameworks/R.framework/Versions/*/Resources/lib/libomp.dylib')
    if r_paths:
        omp_lib_path = r_paths[0]

if omp_lib_path and os.path.exists(omp_lib_path):
    print(f"[OK] Found OpenMP library: {omp_lib_path}")
    
    # Find XGBoost and LightGBM installation paths using sys.path
    import site
    site_packages = []
    
    # Get site-packages from sys.path
    for path in sys.path:
        if 'site-packages' in path:
            site_packages.append(path)
    
    # Also try getsitepackages
    try:
        site_packages.extend(site.getsitepackages())
    except:
        pass
    
    # Remove duplicates and check each path
    site_packages = list(set(site_packages))
    
    # Fix XGBoost library dependency
    xgb_fixed = False
    for sp in site_packages:
        xgb_lib_dir = os.path.join(sp, 'xgboost', 'lib')
        xgb_dylib = os.path.join(xgb_lib_dir, 'libxgboost.dylib')
        omp_in_xgb = os.path.join(xgb_lib_dir, 'libomp.dylib')
        
        if os.path.exists(xgb_dylib):
            # Copy libomp to XGBoost lib directory if not exists
            if not os.path.exists(omp_in_xgb):
                shutil.copy2(omp_lib_path, omp_in_xgb)
                print(f"[OK] Copied libomp.dylib to XGBoost lib directory")
            
            # Update library dependency to use @loader_path
            try:
                result = subprocess.run(['install_name_tool', '-change', 
                                       '@rpath/libomp.dylib', 
                                       '@loader_path/libomp.dylib', 
                                       xgb_dylib], 
                                      capture_output=True, text=True, check=False)
                if result.returncode == 0:
                    print("[OK] Fixed XGBoost library dependency")
                    xgb_fixed = True
            except FileNotFoundError:
                pass
            break
    
    # Fix LightGBM library dependency
    lgb_fixed = False
    for sp in site_packages:
        lgb_lib_dir = os.path.join(sp, 'lightgbm', 'lib')
        lgb_dylib = os.path.join(lgb_lib_dir, 'lib_lightgbm.dylib')
        omp_in_lgb = os.path.join(lgb_lib_dir, 'libomp.dylib')
        
        if os.path.exists(lgb_dylib):
            # Copy libomp to LightGBM lib directory if not exists
            if not os.path.exists(omp_in_lgb):
                shutil.copy2(omp_lib_path, omp_in_lgb)
                print(f"[OK] Copied libomp.dylib to LightGBM lib directory")
            
            # Update library dependency
            try:
                result = subprocess.run(['install_name_tool', '-change', 
                                       '@rpath/libomp.dylib', 
                                       '@loader_path/libomp.dylib', 
                                       lgb_dylib], 
                                      capture_output=True, text=True, check=False)
                if result.returncode == 0:
                    print("[OK] Fixed LightGBM library dependency")
                    lgb_fixed = True
            except FileNotFoundError:
                pass
            break
else:
    print("[WARNING] Warning: Could not find libomp.dylib. XGBoost and LightGBM may not work.")

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, loguniform, randint
import warnings
warnings.filterwarnings('ignore')

# Import XGBoost and LightGBM
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
    print("[OK] XGBoost imported successfully")
except Exception as e:
    print(f"[WARNING] Warning: XGBoost not available: {e}")
    XGBOOST_AVAILABLE = False
    xgb = None

try:
    import lightgbm as lgb
    LIGHTGBM_AVAILABLE = True
    print("[OK] LightGBM imported successfully")
except Exception as e:
    print(f"[WARNING] Warning: LightGBM not available: {e}")
    LIGHTGBM_AVAILABLE = False
    lgb = None

print("="*60)
print("Module 5/5: Model Training & Randomized Search Benchmarking")
print("="*60)

# ============================================================================
# Task 1: Load MICE Datasets
# ============================================================================
print("\n" + "="*60)
print("Task 1: Loading MICE Datasets")
print("="*60)

M = 5
mice_X_datasets = []
mice_Y_datasets = []

for m in range(1, M + 1):
    X_file = f'LISS_Final_X_M{m}.csv'
    Y_file = f'LISS_Final_Y_M{m}.csv'
    
    try:
        X_m = pd.read_csv(X_file, encoding='utf-8', index_col=0)
        Y_m = pd.read_csv(Y_file, encoding='utf-8', index_col=0)
        
        # Align indices
        common_idx = X_m.index.intersection(Y_m.index)
        X_m = X_m.loc[common_idx]
        Y_m = Y_m.loc[common_idx]
        
        # Check for missing values
        X_missing = X_m.isna().sum().sum()
        Y_missing = Y_m.isna().sum().sum()
        
        if X_missing > 0 or Y_missing > 0:
            valid_mask = ~(X_m.isna().any(axis=1) | Y_m.isna().any(axis=1))
            X_m = X_m[valid_mask]
            Y_m = Y_m[valid_mask]
        
        # Extract target variable
        target_col = Y_m.columns[0]
        y_m = Y_m[target_col].values
        X_m_values = X_m.values
        
        mice_X_datasets.append(X_m_values)
        mice_Y_datasets.append(y_m)
        
        print(f"[OK] M{m}: X shape {X_m_values.shape}, y shape {y_m.shape}, target: {target_col}")
        
    except FileNotFoundError as e:
        print(f"[ERROR] M{m}: Could not find {X_file} or {Y_file}")
        break
    except Exception as e:
        print(f"[ERROR] M{m}: Error loading data: {e}")
        break

if len(mice_X_datasets) == M:
    print(f"\n[OK] Successfully loaded {M} MICE datasets")
    print(f"  Total datasets: {len(mice_X_datasets)}")
    print(f"  All datasets ready for modeling")
else:
    print(f"\n[ERROR] Only loaded {len(mice_X_datasets)}/{M} datasets. Cannot proceed.")
    mice_X_datasets = []
    mice_Y_datasets = []

# ============================================================================
# Task 2: Define Model List and Parameter Spaces
# ============================================================================
if len(mice_X_datasets) == M and len(mice_Y_datasets) == M:
    print("\n" + "="*60)
    print("Task 2: Define Models and Randomized Search Parameter Distributions")
    print("="*60)
    
    # Define models and their parameter distributions for RandomizedSearchCV
    models_config = {
        'Elastic-Net': {
            'model': ElasticNet(random_state=42, max_iter=1000),
            'param_distributions': {
                'alpha': loguniform(1e-3, 1e1),  # 0.001 to 10
                'l1_ratio': uniform(0.1, 0.8)  # 0.1 to 0.9
            }
        },
        'SVR': {
            'model': SVR(),
            'param_distributions': {
                'C': loguniform(1e-2, 1e2),  # 0.01 to 100
                'gamma': ['scale', 'auto', 1e-4, 1e-3, 1e-2, 1e-1],  # scale, auto, or numeric values
                'epsilon': uniform(0.01, 0.49)  # 0.01 to 0.5
            }
        },
        'Random Forest': {
            'model': RandomForestRegressor(random_state=42, n_jobs=1),
            'param_distributions': {
                'n_estimators': randint(50, 201),  # 50 to 200
                'max_depth': [5, 10, 15, 20, None],
                'min_samples_split': randint(2, 11)  # 2 to 10
            }
        }
    }
    
    # Add XGBoost
    if XGBOOST_AVAILABLE:
        models_config['XGBoost'] = {
            'model': xgb.XGBRegressor(random_state=42, n_jobs=1, verbosity=0),
            'param_distributions': {
                'n_estimators': randint(50, 201),  # 50 to 200
                'max_depth': randint(3, 8),  # 3 to 7
                'learning_rate': loguniform(1e-3, 1e0),  # 0.001 to 1.0
                'subsample': uniform(0.6, 0.4)  # 0.6 to 1.0
            }
        }
    else:
        raise ImportError("XGBoost is required but could not be imported. Please check OpenMP installation.")
    
    # Add LightGBM
    if LIGHTGBM_AVAILABLE:
        models_config['LightGBM'] = {
            'model': lgb.LGBMRegressor(random_state=42, n_jobs=1, verbosity=-1),
            'param_distributions': {
                'n_estimators': randint(50, 201),  # 50 to 200
                'max_depth': randint(3, 8),  # 3 to 7
                'learning_rate': loguniform(1e-3, 1e0),  # 0.001 to 1.0
                'num_leaves': randint(31, 101)  # 31 to 100
            }
        }
    else:
        raise ImportError("LightGBM is required but could not be imported. Please check OpenMP installation.")
    
    print(f"\nDefined {len(models_config)} models with parameter distributions:")
    for model_name, config in models_config.items():
        n_params = len(config['param_distributions'])
        print(f"  - {model_name}: {n_params} hyperparameters (using distributions)")
    
    # ============================================================================
    # Task 3: MICE Pooling with Nested Cross-Validation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: MICE Pooling with Nested Cross-Validation")
    print("="*60)
    
    # Outer CV: 5-fold for final performance evaluation
    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Inner CV: 3-fold for hyperparameter tuning
    inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
    
    print(f"\nCross-Validation Setup:")
    print(f"  MICE Datasets: {M}")
    print(f"  Outer CV: {outer_cv.n_splits}-fold (for final evaluation)")
    print(f"  Inner CV: {inner_cv.n_splits}-fold RandomizedSearchCV (n_iter=50 for hyperparameter tuning)")
    print(f"  Total test folds: {M} datasets × {outer_cv.n_splits} folds = {M * outer_cv.n_splits} test results")
    
    # Store results for pooling (all RMSE and R² scores across all MICE datasets)
    pooled_results = {}
    
    print(f"\n{'='*60}")
    print("Training and Evaluating Models Across MICE Datasets...")
    print(f"{'='*60}")
    
    # Main loop: iterate over MICE datasets
    for m_idx in range(M):
        print(f"\n{'='*60}")
        print(f"MICE Dataset {m_idx+1}/{M}")
        print(f"{'='*60}")
        
        X = mice_X_datasets[m_idx]
        y = mice_Y_datasets[m_idx]
        
        print(f"  Dataset shape: X {X.shape}, y {y.shape}")
        
                # Outer CV loop
        for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X), 1):
            X_train_outer, X_test_outer = X[train_idx], X[test_idx]
            y_train_outer, y_test_outer = y[train_idx], y[test_idx]

            # Process each model
            for model_name, config in models_config.items():
                # Initialize storage for this model if not already present
                if model_name not in pooled_results:
                    pooled_results[model_name] = {
                        'rmse_scores': []
                    }

                model = config['model']
                param_distributions = config['param_distributions']

                # Inner Randomized Search for hyperparameter tuning
                random_search = RandomizedSearchCV(
                    estimator=model,
                    param_distributions=param_distributions,
                    n_iter=50,  # Number of parameter settings sampled
                    cv=inner_cv,
                    scoring='neg_mean_squared_error',
                    n_jobs=1,  # Use single job to avoid memory issues
                    random_state=42,
                    verbose=0
                )

                try:
                    # Fit on outer training set
                    random_search.fit(X_train_outer, y_train_outer)

                    # Get best model
                    best_model = random_search.best_estimator_

                    # Predict on outer test set
                    y_pred = best_model.predict(X_test_outer)

                    # Calculate RMSE on test set
                    rmse = np.sqrt(mean_squared_error(y_test_outer, y_pred))

                    # Collect results for pooling (across all MICE datasets)
                    pooled_results[model_name]['rmse_scores'].append(rmse)

                    # Verbose logging for first few folds
                    if fold_idx == 1 and m_idx == 0:
                        print(f"\n  {model_name}:")
                    if fold_idx == 1:
                        print(f"    M{m_idx+1} Fold {fold_idx}/{outer_cv.n_splits}: RMSE={rmse:.4f}")
                    elif fold_idx <= 3:
                        print(f"    M{m_idx+1} Fold {fold_idx}/{outer_cv.n_splits}: RMSE={rmse:.4f}")
                    elif fold_idx == outer_cv.n_splits:
                        print(f"    M{m_idx+1} Fold {fold_idx}/{outer_cv.n_splits}: RMSE={rmse:.4f}")

                except Exception as e:
                    # Do not stop the whole process; log the error and continue
                    print(f"[ERROR] Error training {model_name} on M{m_idx+1} Fold {fold_idx}: {e}")
                    continue
    
    # ============================================================================
    # Task 4: MICE Pooling (Result Integration)
    # ============================================================================
    print("\n" + "="*60)
    print("Task 4: MICE Pooling (Result Integration)")
    print("="*60)
    
    # Calculate pooled statistics for each model
    results = {}
    for model_name in pooled_results.keys():
        rmse_scores = pooled_results[model_name]['rmse_scores']
        
        results[model_name] = {
            'mean_rmse': np.mean(rmse_scores),
            'std_rmse': np.std(rmse_scores),
            'rmse_scores': rmse_scores,
            'n_folds': len(rmse_scores)
        }
        
        print(f"\n  {model_name}:")
        print(f"    Total test folds: {len(rmse_scores)}")
        print(f"    Pooled Mean RMSE: {results[model_name]['mean_rmse']:.4f} (±{results[model_name]['std_rmse']:.4f})")
    
    # ============================================================================
    # Task 5: Final Output
    # ============================================================================
    print("\n" + "="*60)
    print("Task 5: Final Results Summary (After MICE Pooling)")
    print("="*60)
    
    # Create results table (sorted by RMSE, ascending)
    print("\n" + "="*80)
    print("Pooled Model Performance: Nested Cross-Validation Across 5 MICE Datasets")
    print("="*80)
    print(f"Total test folds: {M} datasets × {outer_cv.n_splits} folds = {M * outer_cv.n_splits} results per model")
    
    # Sort models by RMSE (ascending - lower is better)
    sorted_results = sorted(results.items(), key=lambda x: x[1]['mean_rmse'])
    
    print(f"\n{'Model':<20} {'Mean RMSE':<15} {'RMSE Std':<15}")
    print("-" * 50)
    
    for model_name, r in sorted_results:
        print(f"{model_name:<20} {r['mean_rmse']:>13.4f} {r['std_rmse']:>14.4f}")
    
    # Determine best model
    print("\n" + "="*80)
    print("Best Model Selection")
    print("="*80)
    
    # Find best model by lowest RMSE
    best_by_rmse = min(results.items(), key=lambda x: x[1]['mean_rmse'])
    
    # Find best model by highest R²
    
    print(f"\nBest Model by RMSE (lowest):")
    print(f"  Model: {best_by_rmse[0]}")
    print(f"  Mean RMSE: {best_by_rmse[1]['mean_rmse']:.4f} (±{best_by_rmse[1]['std_rmse']:.4f})")
    

    
    # Overall best model is the same as best by RMSE (since RMSE is the only metric)
    best_overall = best_by_rmse[0]
    best_result = best_by_rmse[1]
    
    print(f"\n{'='*80}")
    print(f"[BEST] BEST OVERALL MODEL (by RMSE): {best_overall}")
    print(f"{'='*80}")
    print(f"  Mean RMSE: {best_result['mean_rmse']:.4f} (±{best_result['std_rmse']:.4f})")
    
    # Detailed ranking table (by RMSE only)
    print(f"\n{'='*80}")
    print("Model Rankings (by RMSE)")
    print("="*80)
    # Create results DataFrame for saving (sorted by RMSE, ascending)
    results_table_data = []
    for rank, (model_name, r) in enumerate(sorted_results, start=1):
        results_table_data.append({
            'Model': model_name,
            'Mean RMSE': r['mean_rmse'],
            'RMSE Std': r['std_rmse'],
            'Rank': rank
        })
    
    results_df = pd.DataFrame(results_table_data)
    
    # Save results to CSV
    results_filename = 'Model_Benchmarking_RMSE.csv'
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"\n[OK] Results saved: {results_filename}")
    
    print("\n" + "="*60)
    print("[OK] Module 5/5 completed!")
    print("Pooled results are stored in 'results' dictionary")
    print(f"  Processed {M} MICE datasets")
    print(f"  Total test folds per model: {M * outer_cv.n_splits}")
    print(f"  Results saved to: {results_filename}")
    print("="*60)
    
else:
    print("\n[ERROR] Cannot proceed: MICE datasets not loaded successfully.")
    print(f"  Expected {M} datasets, but loaded {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")


[OK] Found OpenMP library: /Library/Frameworks/R.framework/Versions/4.4-arm64/Resources/lib/libomp.dylib
[OK] Fixed XGBoost library dependency
[OK] Fixed LightGBM library dependency
[OK] XGBoost imported successfully
[OK] LightGBM imported successfully
Module 5/5: Model Training & Randomized Search Benchmarking

Task 1: Loading MICE Datasets
[OK] M1: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction
[OK] M2: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction
[OK] M3: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction
[OK] M4: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction
[OK] M5: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction

[OK] Successfully loaded 5 MICE datasets
  Total datasets: 5
  All datasets ready for modeling

Task 2: Define Models and Randomized Search Parameter Distributions

Defined 5 models with parameter distributions:
  - Elastic-Net: 2 hyperparameters (using distributio

In [33]:
# Cell 11:  RQ1 Incremental Contribution Analysis (LightGBM)
# RQ1: Incremental Contribution Analysis using LightGBM
print("="*60)
print("Cell 11: RQ1 Incremental Contribution Analysis (LightGBM)")
print("="*60)

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import matplotlib.pyplot as plt  # Temporarily kept for existing visualization code
import warnings
warnings.filterwarnings('ignore')

# Check if LightGBM is available
try:
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    raise ImportError("LightGBM is required but could not be imported.")

# Set matplotlib style (temporarily kept for existing visualization code)
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 12

# ============================================================================
# ============================================================================
# Task 1: Environment Verification & Data Loading
# ============================================================================
print("\n" + "="*60)
print("Task 1: Environment Verification & Data Loading")
print("="*60)

# Case A: Check if mice_X_datasets and mice_Y_datasets exist in environment
try:
    if 'mice_X_datasets' in globals() and 'mice_Y_datasets' in globals() and \
       mice_X_datasets is not None and mice_Y_datasets is not None and \
       len(mice_X_datasets) > 0 and len(mice_Y_datasets) > 0:
        print("[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment")
        print(f"  Found {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")
        M = len(mice_X_datasets)
        # Ensure mice_X_dataframes exists
        if 'mice_X_dataframes' not in globals() or mice_X_dataframes is None or len(mice_X_dataframes) == 0:
            print("[WARNING] mice_X_dataframes not found, will reload to preserve feature names")
            raise NameError("mice_X_dataframes missing")
except (NameError, AttributeError):
    # Case B: Reload from CSV files
    print("[INFO] Case B: Reloading MICE datasets from CSV files")
    data_loaded = False

if not data_loaded:
    M = 5
    mice_X_datasets = []
    mice_Y_datasets = []
    mice_X_dataframes = []

    for m in range(1, M + 1):
        X_file = f'LISS_Final_X_M{m}.csv'
        Y_file = f'LISS_Final_Y_M{m}.csv'
        
        try:
            X_m = pd.read_csv(X_file, encoding='utf-8', index_col=0)
            Y_m = pd.read_csv(Y_file, encoding='utf-8', index_col=0)
            
            # Align indices
            common_idx = X_m.index.intersection(Y_m.index)
            X_m = X_m.loc[common_idx]
            Y_m = Y_m.loc[common_idx]
            
            # Check for missing values
            X_missing = X_m.isna().sum().sum()
            Y_missing = Y_m.isna().sum().sum()
            
            if X_missing > 0 or Y_missing > 0:
                valid_mask = ~(X_m.isna().any(axis=1) | Y_m.isna().any(axis=1))
                X_m = X_m[valid_mask]
                Y_m = Y_m[valid_mask]
            
            # Extract target variable
            target_col = Y_m.columns[0]
            y_m = Y_m[target_col].values
            X_m_values = X_m.values
            
            mice_X_datasets.append(X_m_values)
            mice_Y_datasets.append(y_m)
            mice_X_dataframes.append(X_m.copy())  # Keep DataFrame for feature names
            
            print(f"[OK] M{m}: X shape {X_m_values.shape}, y shape {y_m.shape}, target: {target_col}")
            
        except FileNotFoundError as e:
            print(f"[ERROR] M{m}: Could not find {X_file} or {Y_file}")
            break
        except Exception as e:
            print(f"[ERROR] M{m}: Error loading data: {e}")
            break

    if len(mice_X_datasets) == M:
        print(f"\n[OK] Successfully loaded {M} MICE datasets from CSV files")
        print(f"  Total datasets: {len(mice_X_datasets)}")
        print(f"  Feature names preserved for block selection")
        mice_X_dataframes = []

# Verify data is available
if len(mice_X_datasets) == M and len(mice_Y_datasets) == M:
    print(f"\n[OK] Successfully loaded {M} MICE datasets")
    print(f"  Total datasets: {len(mice_X_datasets)}")
    print(f"  All datasets ready for visualization")

# ============================================================================
# Task 2: Define Model
# ============================================================================
if len(mice_X_datasets) == M and len(mice_Y_datasets) == M:
    print("\n" + "="*60)
    print("Task 2: Define Random Forest Model")
    print("="*60)
    
        # Define LightGBM Regressor
    lgb_model = lgb.LGBMRegressor(
        n_estimators=100,
        max_depth=15,
        learning_rate=0.1,
        num_leaves=31,
        min_child_samples=20,
        random_state=42,
        n_jobs=1,
        verbosity=-1
    )

    print(f"\nLightGBM Configuration:")
    print(f"  n_estimators: 100")
    print(f"  max_depth: 15")
    print(f"  learning_rate: 0.1")
    print(f"  num_leaves: 31")
    print(f"  min_child_samples: 20")
    print(f"  random_state: 42")
    print(f"  n_jobs: 1")
    print(f"  verbosity: -1")
    
    print(f"\nEvaluation Metric:")
    print(f"  RMSE (Root Mean Squared Error) - to measure prediction error")

# ============================================================================
# Task 3: Define 4 Incremental Feature Blocks
# ============================================================================
    print("\n" + "="*60)
    print("Task 3: Define 4 Incremental Feature Blocks")
    print("="*60)
    
    # Get feature names from first dataset
    feature_names = mice_X_dataframes[0].columns.tolist()
    
    # Define feature blocks
    feature_blocks = {}
    
    # Block 1: Baseline Controls
    # Age, Children Count, Self-rated Health, Gender OHE, Education OHE, Income OHE
    block1_features = []
    
    # Continuous baseline controls
    baseline_continuous = ['cw23p003', 'cf23p455', 'ch23p004']  # Age, Children Count, Self-rated Health
    for feat in baseline_continuous:
        if feat in feature_names:
            block1_features.append(feat)
    
    # Gender OHE dummies
    gender_dummies = [f for f in feature_names if f.startswith('ch23p001_')]
    block1_features.extend(gender_dummies)
    
    # Education OHE dummies
    education_dummies = [f for f in feature_names if f.startswith('Education_Recode_')]
    block1_features.extend(education_dummies)
    
    # Income OHE dummies
    income_dummies = [f for f in feature_names if f.startswith('Income_Quartile_')]
    block1_features.extend(income_dummies)
    
    feature_blocks['Block 1'] = {
        'name': 'Baseline Controls',
        'features': block1_features,
        'description': 'Age, Children Count, Self-rated Health, Gender OHE, Education OHE, Income OHE'
    }
    
    # Block 2: Block 1 + Work Characteristics
    block2_features = block1_features.copy()
    work_char_features = ['Delta_Autonomy', 'Delta_Workload', 'Delta_Skills']
    for feat in work_char_features:
        if feat in feature_names:
            block2_features.append(feat)
    
    feature_blocks['Block 2'] = {
        'name': '+ Work Characteristics',
        'features': block2_features,
        'description': 'Block 1 + Δ Autonomy, Δ Workload, Δ Skills'
    }
    
    # Block 3: Block 2 + WLB & Health Factors
    block3_features = block2_features.copy()
    wlb_health_features = ['Delta_RemoteHours', 'Delta_WFC_Proxy', 'Delta_Health_Hindrance']
    for feat in wlb_health_features:
        if feat in feature_names:
            block3_features.append(feat)
    
    feature_blocks['Block 3'] = {
        'name': '+ WLB & Health Factors',
        'features': block3_features,
        'description': 'Block 2 + Δ RemoteHours, Δ WFC Proxy, Δ Health Hindrance'
    }
    
    # Block 4: Block 3 + Personality Traits (Full Model)
    block4_features = block3_features.copy()
    personality_features = ['N_Score', 'E_Score', 'C_Score', 'A_Score', 'O_Score']
    for feat in personality_features:
        if feat in feature_names:
            block4_features.append(feat)
    
    feature_blocks['Block 4'] = {
        'name': '+ Personality Traits',
        'features': block4_features,
        'description': 'Block 3 + N_Score, E_Score, C_Score, A_Score, O_Score (Full Model)'
    }
    
    # Print feature block definitions
    print(f"\nFeature Blocks Definition:")
    print("="*80)
    for block_name, block_info in feature_blocks.items():
        available_features = [f for f in block_info['features'] if f in feature_names]
        print(f"\n{block_name}: {block_info['name']}")
        print(f"  Description: {block_info['description']}")
        print(f"  Features: {len(available_features)}/{len(block_info['features'])} available")
        if len(available_features) <= 10:
            print(f"    {', '.join(available_features)}")
# ============================================================================
# Task 4: Execute Incremental Modeling
# ============================================================================
    print("\n" + "="*60)
    print("Task 4: Execute Incremental Modeling")
    print("="*60)
    
    # 5-Fold Cross-Validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Store RMSE scores for each block across all MICE datasets and CV folds
    block_rmse = {block_name: [] for block_name in feature_blocks.keys()}
    
    print(f"\nCross-Validation Setup:")
    print(f"  CV: {cv.n_splits}-fold KFold")
    print(f"  MICE Datasets: {M}")
    print(f"  Feature Blocks: {len(feature_blocks)}")
    print(f"  Total evaluations: {M} datasets × {len(feature_blocks)} blocks × {cv.n_splits} folds")
    
    print(f"\n{'='*60}")
    print("Training Models Across Feature Blocks and MICE Datasets...")
    print(f"{'='*60}")
    
    # Main loop: iterate over MICE datasets
    for m_idx in range(M):
        print(f"\nProcessing MICE Dataset {m_idx+1}/{M}...")
        
        X_df = mice_X_dataframes[m_idx]
        y = mice_Y_datasets[m_idx]
        
        print(f"  Dataset shape: X {X_df.shape}, y {y.shape}")
        
        # Process each feature block
        for block_name, block_info in feature_blocks.items():
            # Select features for this block
            available_block_features = [f for f in block_info['features'] if f in X_df.columns]
            
            if len(available_block_features) == 0:
                print(f"    [WARNING] {block_name}: No features available, skipping")
                continue
            
            X_block = X_df[available_block_features].values
            
            print(f"    {block_name}: {len(available_block_features)} features")
            
            # Cross-validation (using neg_mean_squared_error, then convert to RMSE)
            neg_mse_scores = cross_val_score(
                lgb_model,
                X_block,
                y,
                cv=cv,
                scoring='neg_mean_squared_error',
                n_jobs=1
            )
            
            # Convert to RMSE (neg_mean_squared_error is negative, so we negate and take sqrt)
            rmse_scores = np.sqrt(-neg_mse_scores)
            
            # Store RMSE scores
            block_rmse[block_name].extend(rmse_scores)
            
            mean_rmse = np.mean(rmse_scores)
            std_rmse = np.std(rmse_scores)
            print(f"      Mean RMSE: {mean_rmse:.4f} (±{std_rmse:.4f})")
    
    print(f"\n[OK] Incremental modeling completed for all {M} MICE datasets")

# ============================================================================
# Task 5: Results Pooling and Output
# ============================================================================
    print("\n" + "="*60)
    print("Task 5: Results Pooling and Output")
    print("="*60)
    
    # Calculate pooled RMSE (average across all MICE datasets and CV folds)
    pooled_rmse = {}
    pooled_rmse_std = {}
    pooled_rmse_count = {}
    
    for block_name in feature_blocks.keys():
        if len(block_rmse[block_name]) > 0:
            pooled_rmse[block_name] = np.mean(block_rmse[block_name])
            pooled_rmse_std[block_name] = np.std(block_rmse[block_name])
            pooled_rmse_count[block_name] = len(block_rmse[block_name])
    
    # Calculate cumulative RMSE and Error Reduction (ΔRMSE)
    cumulative_rmse = {}
    delta_rmse = {}
    
    # Block 1 is the baseline
    cumulative_rmse['Block 1'] = pooled_rmse['Block 1']
    delta_rmse['Block 1'] = 0.0  # First block has no previous block for comparison
    
    # Calculate for subsequent blocks
    # Error Reduction (ΔRMSE) = Previous Block RMSE - Current Block RMSE
    # Positive value means error reduction (improvement), negative means error increase (worsening)
    for i in range(2, len(feature_blocks) + 1):
        block_name = f'Block {i}'
        prev_block_name = f'Block {i-1}'
        
        cumulative_rmse[block_name] = pooled_rmse[block_name]
        if not np.isnan(pooled_rmse[block_name]) and not np.isnan(cumulative_rmse[prev_block_name]):
            # Error Reduction = Previous RMSE - Current RMSE
            delta_rmse[block_name] = cumulative_rmse[prev_block_name] - pooled_rmse[block_name]
    results_data = []
    for block_name in feature_blocks.keys():
        results_data.append({
            'Model Block': block_name,
            'Included Features': feature_blocks[block_name]['name'],
            'Cumulative RMSE (Pooled Mean)': cumulative_rmse[block_name] if block_name in cumulative_rmse else pooled_rmse[block_name],
            'Error Reduction (ΔRMSE)': delta_rmse[block_name] if block_name in delta_rmse else np.nan,
            'N_Evaluations': pooled_rmse_count[block_name] if block_name in pooled_rmse_count else 0
        })
    
    results_df = pd.DataFrame(results_data)
    
    # Print results table
    print("\n" + "="*100)
    print("RQ1: Incremental Contribution Analysis Results (RMSE)")
    print("="*100)
    print(f"\n{'Model Block':<12} {'Included Features':<35} {'Cumulative RMSE (Pooled Mean)':<30} {'Error Reduction (ΔRMSE)':<25}")
    print("-"*100)
    
    for idx, row in results_df.iterrows():
        cum_rmse_str = f"{row['Cumulative RMSE (Pooled Mean)']:.4f}" if not np.isnan(row['Cumulative RMSE (Pooled Mean)']) else "N/A"
        delta_rmse_str = f"{row['Error Reduction (ΔRMSE)']:+.4f}" if not np.isnan(row['Error Reduction (ΔRMSE)']) else "N/A"
        print(f"{row['Model Block']:<12} {row['Included Features']:<35} {cum_rmse_str:>29} {delta_rmse_str:>24}")
    
    # Print detailed statistics
    print("\n" + "="*100)
    print("Detailed Statistics:")
    print("="*100)
    print(f"\n{'Model Block':<12} {'Included Features':<35} {'Cumulative RMSE':<20} {'RMSE Std':<15} {'Error Reduction (ΔRMSE)':<25} {'N_Evaluations':<15}")
    print("-"*100)
    
    for idx, row in results_df.iterrows():
        cum_rmse_str = f"{row['Cumulative RMSE (Pooled Mean)']:.4f}" if not np.isnan(row['Cumulative RMSE (Pooled Mean)']) else "N/A"
        # Get std from pooled_rmse_std
        block_name = row['Model Block']
        rmse_std_str = f"±{pooled_rmse_std[block_name]:.4f}" if block_name in pooled_rmse_std and not np.isnan(pooled_rmse_std[block_name]) else "N/A"
        delta_rmse_str = f"{row['Error Reduction (ΔRMSE)']:+.4f}" if not np.isnan(row['Error Reduction (ΔRMSE)']) else "N/A"
        print(f"{row['Model Block']:<12} {row['Included Features']:<35} {cum_rmse_str:>19} {rmse_std_str:>14} {delta_rmse_str:>24} {row['N_Evaluations']:>14}")
    
    # Error Reduction Analysis
    print("\n" + "="*100)
    print("Error Reduction Analysis:")
    print("="*100)
    
    if not np.isnan(pooled_rmse['Block 1']):
        baseline_rmse = pooled_rmse['Block 1']
        print(f"\nBaseline RMSE (Block 1): {baseline_rmse:.4f}")
        print(f"\nError Reduction Breakdown:")
        
        for block_name in feature_blocks.keys():
            if block_name in delta_rmse and not np.isnan(delta_rmse[block_name]):
                if delta_rmse[block_name] > 0:
                    interpretation = "error reduction (improvement)"
                elif delta_rmse[block_name] < 0:
                    interpretation = "error increase (worsening)"
    # Save results
    results_filename = 'RQ1_Incremental_RMSE_LightGBM.csv'
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"\n[OK] Results saved: {results_filename}")
    
    print(f"\n{'='*60}")
    print("[OK] RQ1 Incremental Contribution Analysis completed!")
    print(f"  Processed {M} MICE datasets")
    print(f"  Analyzed {len(feature_blocks)} feature blocks")
    print(f"  Generated incremental contribution analysis table")
    print("="*60)

else:
    print("\n[ERROR] Cannot proceed: MICE datasets not loaded successfully.")
    print(f"  Expected {M} datasets, but loaded {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")


Cell 11: RQ1 Incremental Contribution Analysis (LightGBM)

Task 1: Environment Verification & Data Loading
[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment
  Found 5 X datasets and 5 Y datasets

[OK] Successfully loaded 5 MICE datasets
  Total datasets: 5
  All datasets ready for visualization

Task 2: Define Random Forest Model

LightGBM Configuration:
  n_estimators: 100
  max_depth: 15
  learning_rate: 0.1
  num_leaves: 31
  min_child_samples: 20
  random_state: 42
  n_jobs: 1
  verbosity: -1

Evaluation Metric:
  RMSE (Root Mean Squared Error) - to measure prediction error

Task 3: Define 4 Incremental Feature Blocks

Feature Blocks Definition:

Block 1: Baseline Controls
  Description: Age, Children Count, Self-rated Health, Gender OHE, Education OHE, Income OHE
  Features: 11/11 available

Block 2: + Work Characteristics
  Description: Block 1 + Δ Autonomy, Δ Workload, Δ Skills
  Features: 14/14 available

Block 3: + WLB & Health Factors
  Descri

In [1]:
# Cell 12:  Pre-RQ2 - SHAP Calculation & Feature Extraction (LightGBM)
# Pre-RQ2: SHAP Calculation & Feature Extraction using LightGBM
print("="*60)
print("Cell 12: Pre-RQ2 - SHAP Calculation & Feature Extraction (LightGBM)")
print("="*60)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
import lightgbm as lgb
import time
import warnings
warnings.filterwarnings('ignore')

# Check if LightGBM is available
try:
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    raise ImportError("LightGBM is required but could not be imported.")

# Set matplotlib style
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# ============================================================================
# Task 1: Environment Verification & Data Loading
# ============================================================================
print("\n" + "="*60)
print("Task 1: Environment Verification & Data Loading")
print("="*60)

# Case A: Check if mice_X_datasets and mice_Y_datasets exist in environment
try:
    if 'mice_X_datasets' in globals() and 'mice_Y_datasets' in globals() and \
       mice_X_datasets is not None and mice_Y_datasets is not None and \
       len(mice_X_datasets) > 0 and len(mice_Y_datasets) > 0:
        print("[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment")
        print(f"  Found {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")
        M = len(mice_X_datasets)
        # Ensure mice_X_dataframes exists
        if 'mice_X_dataframes' not in globals() or mice_X_dataframes is None or len(mice_X_dataframes) == 0:
            print("[WARNING] mice_X_dataframes not found, will reload to preserve feature names")
            raise NameError("mice_X_dataframes missing")
        else:
            print(f"  Found {len(mice_X_dataframes)} X dataframes with feature names")
            data_loaded = True
    else:
        raise NameError("Variables not available")
except (NameError, AttributeError):
    # Case B: Reload from CSV files
    print("[INFO] Case B: Reloading MICE datasets from CSV files")
    data_loaded = False

if not data_loaded:
    M = 5
    mice_X_datasets = []
    mice_Y_datasets = []
    mice_X_dataframes = []

    for m in range(1, M + 1):
        X_file = f'LISS_Final_X_M{m}.csv'
        Y_file = f'LISS_Final_Y_M{m}.csv'
        
        try:
            X_m = pd.read_csv(X_file, encoding='utf-8', index_col=0)
            Y_m = pd.read_csv(Y_file, encoding='utf-8', index_col=0)
            
            # Align indices
            common_idx = X_m.index.intersection(Y_m.index)
            X_m = X_m.loc[common_idx]
            Y_m = Y_m.loc[common_idx]
            
            # Check for missing values
            X_missing = X_m.isna().sum().sum()
            Y_missing = Y_m.isna().sum().sum()
            
            if X_missing > 0 or Y_missing > 0:
                valid_mask = ~(X_m.isna().any(axis=1) | Y_m.isna().any(axis=1))
                X_m = X_m[valid_mask]
                Y_m = Y_m[valid_mask]
            
            # Extract target variable
            target_col = Y_m.columns[0]
            y_m = Y_m[target_col].values
            X_m_values = X_m.values
            
            mice_X_datasets.append(X_m_values)
            mice_Y_datasets.append(y_m)
            mice_X_dataframes.append(X_m.copy())  # Keep DataFrame for feature names
            
            print(f"[OK] M{m}: X shape {X_m_values.shape}, y shape {y_m.shape}, target: {target_col}")
            
        except FileNotFoundError as e:
            print(f"[ERROR] M{m}: Could not find {X_file} or {Y_file}")
            break
        except Exception as e:
            print(f"[ERROR] M{m}: Error loading data: {e}")
            break

    if len(mice_X_datasets) == M:
        print(f"\n[OK] Successfully loaded {M} MICE datasets from CSV files")
        print(f"  Total datasets: {len(mice_X_datasets)}")
        print(f"  Feature names preserved for SHAP analysis")
    else:
        print(f"\n[ERROR] Only loaded {len(mice_X_datasets)}/{M} datasets. Cannot proceed.")
        mice_X_datasets = []
        mice_Y_datasets = []
        mice_X_dataframes = []

# Verify data is available
if len(mice_X_datasets) == M and len(mice_Y_datasets) == M:
    # ============================================================================
    # Task 2: Model Retraining and SHAP Calculation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Model Retraining and SHAP Calculation")
    print("="*60)
    
    # Define LightGBM Regressor
    lgb_model = lgb.LGBMRegressor(
        n_estimators=100,
        max_depth=15,
        learning_rate=0.1,
        num_leaves=31,
        min_child_samples=20,
        random_state=42,
        n_jobs=1,
        verbosity=-1
    )
    
    print(f"\nLightGBM Configuration:")
    print(f"  n_estimators: 100")
    print(f"  max_depth: 15")
    print(f"  learning_rate: 0.1")
    print(f"  num_leaves: 31")
    print(f"  min_child_samples: 20")
    print(f"  random_state: 42")
    print(f"  n_jobs: 1")
    print(f"  verbosity: -1")
    
    # Store SHAP values and Mean Absolute SHAP from all MICE datasets
    shap_values_list = []
    mean_abs_shap_list = []  # Q_m for each MICE dataset
    feature_names = None
    
    print(f"\n{'='*60}")
    print("Training Models and Computing SHAP Values...")
    print(f"{'='*60}")
    
    # Main loop: iterate over MICE datasets
    for m_idx in range(M):
        print(f"\nProcessing MICE Dataset {m_idx+1}/{M}...")
        
        X = mice_X_datasets[m_idx]
        y = mice_Y_datasets[m_idx]
        X_df = mice_X_dataframes[m_idx]
        
        print(f"  Dataset shape: X {X.shape}, y {y.shape}")
        
        # Train LightGBM model on this dataset
        lgb_model.fit(X, y)
        print(f"  [OK] Model trained")
        
        # Get feature names (use from first dataset)
        if feature_names is None:
            feature_names = X_df.columns.tolist()
            print(f"  [OK] Feature names extracted: {len(feature_names)} features")
        
        # Calculate SHAP values using TreeExplainer (optimized for speed)
        print(f"  Computing SHAP values (optimized: using booster_ and check_additivity=False)...")
        start_time = time.time()
        
        # Use booster_ for faster computation and disable additivity check
        try:
            explainer = shap.TreeExplainer(lgb_model.booster_)
            shap_values = explainer.shap_values(X, check_additivity=False)
        except AttributeError:
            # Fallback: if booster_ is not available, use model directly
            print(f"    [WARNING] booster_ not available, using model directly")
            explainer = shap.TreeExplainer(lgb_model)
            shap_values = explainer.shap_values(X, check_additivity=False)
        
        elapsed_time = time.time() - start_time
        print(f"  [OK] SHAP values computed: shape {shap_values.shape} (took {elapsed_time:.2f} seconds)")
        
        # Calculate Mean Absolute SHAP values for each feature (Q_m)
        mean_abs_shap = np.abs(shap_values).mean(axis=0)  # Shape: (n_features,)
        mean_abs_shap_list.append(mean_abs_shap)
        shap_values_list.append(shap_values)
    
    print(f"\n[OK] SHAP values computed for all {M} MICE datasets")
    
    # ============================================================================
    # Task 3: SHAP Pooling and Feature Importance Calculation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: SHAP Pooling and Feature Importance Calculation")
    print("="*60)
    
    # Pool SHAP values (average across MICE datasets)
    pooled_shap_values = np.mean(shap_values_list, axis=0)
    print(f"  Pooled SHAP values shape: {pooled_shap_values.shape}")
    
    # Calculate pooled Mean Absolute SHAP (simple average)
    pooled_mean_abs_shap = np.mean(mean_abs_shap_list, axis=0)
    
    # Create feature importance DataFrame
    feature_importance_data = []
    for i, feat_name in enumerate(feature_names):
        feature_importance_data.append({
            'Feature': feat_name,
            'Mean_Abs_SHAP': pooled_mean_abs_shap[i]
        })
    
    feature_importance_df = pd.DataFrame(feature_importance_data)
    feature_importance_df = feature_importance_df.sort_values('Mean_Abs_SHAP', ascending=False)
    feature_importance_df['Rank'] = range(1, len(feature_importance_df) + 1)
    feature_importance_df = feature_importance_df[['Rank', 'Feature', 'Mean_Abs_SHAP']]
    
    print(f"\n[OK] Feature importance calculated (pooled across {M} MICE datasets)")
    print(f"\nTop 10 Most Important Features:")
    print("-"*60)
    for idx, row in feature_importance_df.head(10).iterrows():
        print(f"  {row['Rank']}. {row['Feature']:<35} (Mean Abs SHAP: {row['Mean_Abs_SHAP']:.6f})")
    
    # ============================================================================
    # Task 4: Rubin's Rule Pooling for Top 10 Features
    # ============================================================================
    print("\n" + "="*60)
    print("Task 4: Rubin's Rule Pooling for Top 10 Features")
    print("="*60)
    
    # Get top 10 features
    top_10_features = feature_importance_df.head(10)['Feature'].tolist()
    print(f"\nTop 10 Features for Rubin's Rule Pooling:")
    for i, feat in enumerate(top_10_features, 1):
        print(f"  {i}. {feat}")
    
    # Extract Q_m (Mean Absolute SHAP) for top 10 features from each MICE dataset
    Q_m_top10 = {}  # Q_m[feature_name] = [Q_m1, Q_m2, ..., Q_m5]
    U_m_top10 = {}  # U_m[feature_name] = [U_m1, U_m2, ..., U_m5]
    
    for feat_name in top_10_features:
        feat_idx = feature_names.index(feat_name)
        Q_m_list = []
        U_m_list = []
        
        for m_idx in range(M):
            # Q_m: Mean Absolute SHAP for this feature in this MICE dataset
            Q_m = mean_abs_shap_list[m_idx][feat_idx]
            Q_m_list.append(Q_m)
            
            # U_m: Variance of absolute SHAP values for this feature in this MICE dataset
            abs_shap_feat = np.abs(shap_values_list[m_idx][:, feat_idx])
            U_m = np.var(abs_shap_feat)  # Variance across samples
            U_m_list.append(U_m)
        
        Q_m_top10[feat_name] = Q_m_list
        U_m_top10[feat_name] = U_m_list
    
    # Apply Rubin's Rule for each top 10 feature
    rubin_results = []
    
    for feat_name in top_10_features:
        Q_m = np.array(Q_m_top10[feat_name])  # Shape: (M,)
        U_m = np.array(U_m_top10[feat_name])  # Shape: (M,)
        
        # Pooled Estimate: Q_bar = mean(Q_m)
        Q_bar = np.mean(Q_m)
        
        # Between-imputation variance: B = var(Q_m)
        B = np.var(Q_m, ddof=1)  # Sample variance
        
        # Within-imputation variance: U_bar = mean(U_m)
        U_bar = np.mean(U_m)
        
        # Total variance: T = U_bar + (1 + 1/M) * B
        T = U_bar + (1 + 1/M) * B
        
        # Pooled Standard Error: SE_pooled = sqrt(T)
        SE_pooled = np.sqrt(T)
        
        rubin_results.append({
            'Feature': feat_name,
            'Pooled_Mean_Abs_SHAP': Q_bar,
            'Pooled_SE': SE_pooled,
            'Between_Variance': B,
            'Within_Variance': U_bar,
            'Total_Variance': T
        })
    
    # Create Rubin's Rule results DataFrame
    rubin_df = pd.DataFrame(rubin_results)
    rubin_df = rubin_df.sort_values('Pooled_Mean_Abs_SHAP', ascending=False)
    rubin_df['Rank'] = range(1, len(rubin_df) + 1)
    rubin_df = rubin_df[['Rank', 'Feature', 'Pooled_Mean_Abs_SHAP', 'Pooled_SE', 'Between_Variance', 'Within_Variance', 'Total_Variance']]
    
    # Print results
    print("\n" + "="*100)
    print("Rubin's Rule Pooling Results (Top 10 Features)")
    print("="*100)
    print(f"\n{'Rank':<6} {'Feature':<35} {'Pooled Mean Abs SHAP':<25} {'Pooled SE':<15}")
    print("-"*100)
    
    for idx, row in rubin_df.iterrows():
        print(f"{row['Rank']:<6} {row['Feature']:<35} {row['Pooled_Mean_Abs_SHAP']:>24.6f} {row['Pooled_SE']:>14.6f}")
    
    # Save results
    results_filename = 'Rubins_Rule_SHAP_LightGBM.csv'
    rubin_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"\n[OK] Results saved: {results_filename}")
    
    # ============================================================================
    # Task 5: Extract and Print Top 10 Feature Names
    # ============================================================================
    print("\n" + "="*60)
    print("Task 5: Extract Top 10 Feature Names")
    print("="*60)
    
    top_10_feature_names = rubin_df['Feature'].tolist()
    
    print(f"\n[OK] Top 10 Most Important Features (for RQ2 Refined Model):")
    print("-"*60)
    for i, feat_name in enumerate(top_10_feature_names, 1):
        feat_importance = rubin_df[rubin_df['Feature'] == feat_name]['Pooled_Mean_Abs_SHAP'].values[0]
        print(f"  {i}. {feat_name:<35} (Pooled Mean Abs SHAP: {feat_importance:.6f})")
    
    print(f"\n[INFO] These top 10 features will be used in Cell 13 (RQ2 Refined Model Comparison)")
    
    # Save top 10 feature names to a simple text file for easy access
    with open('Top_10_Features_LightGBM.txt', 'w', encoding='utf-8') as f:
        for feat_name in top_10_feature_names:
            f.write(f"{feat_name}\n")
    print(f"\n[OK] Top 10 feature names saved to: Top_10_Features_LightGBM.txt")
    
    print(f"\n{'='*60}")
    print("[OK] Pre-RQ2 SHAP Calculation & Feature Extraction completed!")
    print(f"  Processed {M} MICE datasets")
    print(f"  Computed and pooled SHAP values")
    print(f"  Applied Rubin's Rule for top 10 features")
    print(f"  Extracted top 10 feature names for RQ2")
    print("="*60)

else:
    print("\n[ERROR] Cannot proceed: MICE datasets not loaded successfully.")
    print(f"  Expected {M} datasets, but loaded {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")



Cell 12: Pre-RQ2 - SHAP Calculation & Feature Extraction (LightGBM)

Task 1: Environment Verification & Data Loading
[INFO] Case B: Reloading MICE datasets from CSV files
[OK] M1: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction
[OK] M2: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction
[OK] M3: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction
[OK] M4: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction
[OK] M5: X shape (12939, 22), y shape (12939,), target: Delta_JobSatisfaction

[OK] Successfully loaded 5 MICE datasets from CSV files
  Total datasets: 5
  Feature names preserved for SHAP analysis

Task 2: Model Retraining and SHAP Calculation

LightGBM Configuration:
  n_estimators: 100
  max_depth: 15
  learning_rate: 0.1
  num_leaves: 31
  min_child_samples: 20
  random_state: 42
  n_jobs: 1
  verbosity: -1

Training Models and Computing SHAP Values...

Processing MICE Dataset 1/5...
  Dataset shape: X (129

In [3]:
# Cell 13:  RQ2 - Refined Model Comparison (LightGBM)
# RQ2: Refined Model Comparison using LightGBM
print("="*60)
print("Cell 13: RQ2 - Refined Model Comparison (LightGBM)")
print("="*60)

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Check if LightGBM is available
try:
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    raise ImportError("LightGBM is required but could not be imported.")

# ============================================================================
# Task 1: Load Data and SHAP Rankings
# ============================================================================
print("\n" + "="*60)
print("Task 1: Load Data and SHAP Rankings")
print("="*60)

# Load SHAP importance rankings
shap_importance_file = 'Rubins_Rule_SHAP_LightGBM.csv'
try:
    shap_importance_df = pd.read_csv(shap_importance_file, encoding='utf-8')
    print(f"[OK] Loaded SHAP importance rankings from: {shap_importance_file}")
    print(f"  Total features in rankings: {len(shap_importance_df)}")
    
    # Extract top 10 feature names
    top_10_features = shap_importance_df.head(10)['Feature'].tolist()
    print(f"\n[OK] Extracted top 10 features:")
    for i, feat in enumerate(top_10_features, 1):
        print(f"  {i}. {feat}")
        
except FileNotFoundError:
    print(f"[ERROR] Could not find {shap_importance_file}")
    print(f"  Please run Cell 12 first to generate SHAP importance rankings.")
    top_10_features = []

# Load MICE datasets
print(f"\n[INFO] Loading MICE datasets...")

# Case A: Check if mice_X_datasets exist in environment
try:
    if 'mice_X_datasets' in globals() and 'mice_Y_datasets' in globals() and \
       mice_X_datasets is not None and mice_Y_datasets is not None and \
       len(mice_X_datasets) > 0 and len(mice_Y_datasets) > 0:
        print("[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment")
        print(f"  Found {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")
        M = len(mice_X_datasets)
        if 'mice_X_dataframes' not in globals() or mice_X_dataframes is None or len(mice_X_dataframes) == 0:
            print("[WARNING] mice_X_dataframes not found, will reload to preserve feature names")
            raise NameError("mice_X_dataframes missing")
        else:
            print(f"  Found {len(mice_X_dataframes)} X dataframes with feature names")
            data_loaded = True
    else:
        raise NameError("Variables not available")
except (NameError, AttributeError):
    # Case B: Reload from CSV files
    print("[INFO] Case B: Reloading MICE datasets from CSV files")
    data_loaded = False

if not data_loaded:
    M = 5
    mice_X_datasets = []
    mice_Y_datasets = []
    mice_X_dataframes = []

    for m in range(1, M + 1):
        X_file = f'LISS_Final_X_M{m}.csv'
        Y_file = f'LISS_Final_Y_M{m}.csv'
        
        try:
            X_m = pd.read_csv(X_file, encoding='utf-8', index_col=0)
            Y_m = pd.read_csv(Y_file, encoding='utf-8', index_col=0)
            
            # Align indices
            common_idx = X_m.index.intersection(Y_m.index)
            X_m = X_m.loc[common_idx]
            Y_m = Y_m.loc[common_idx]
            
            # Check for missing values
            X_missing = X_m.isna().sum().sum()
            Y_missing = Y_m.isna().sum().sum()
            
            if X_missing > 0 or Y_missing > 0:
                valid_mask = ~(X_m.isna().any(axis=1) | Y_m.isna().any(axis=1))
                X_m = X_m[valid_mask]
                Y_m = Y_m[valid_mask]
            
            # Extract target variable
            target_col = Y_m.columns[0]
            y_m = Y_m[target_col].values
            X_m_values = X_m.values
            
            mice_X_datasets.append(X_m_values)
            mice_Y_datasets.append(y_m)
            mice_X_dataframes.append(X_m.copy())
            
            print(f"[OK] M{m}: X shape {X_m_values.shape}, y shape {y_m.shape}, target: {target_col}")
            
        except FileNotFoundError as e:
            print(f"[ERROR] M{m}: Could not find {X_file} or {Y_file}")
            break
        except Exception as e:
            print(f"[ERROR] M{m}: Error loading data: {e}")
            break

    if len(mice_X_datasets) == M:
        print(f"\n[OK] Successfully loaded {M} MICE datasets from CSV files")
    else:
        print(f"\n[ERROR] Only loaded {len(mice_X_datasets)}/{M} datasets. Cannot proceed.")
        mice_X_datasets = []
        mice_Y_datasets = []
        mice_X_dataframes = []

# Verify data is available
if len(mice_X_datasets) == M and len(mice_Y_datasets) == M and len(top_10_features) == 10:
    # ============================================================================
    # Task 2: Train Refined Model (Top 10 Features)
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Train Refined Model (Top 10 Features)")
    print("="*60)
    
    # Define LightGBM Regressor (same parameters as Cell 11)
    lgb_model = lgb.LGBMRegressor(
        n_estimators=100,
        max_depth=15,
        learning_rate=0.1,
        num_leaves=31,
        min_child_samples=20,
        random_state=42,
        n_jobs=1,
        verbosity=-1
    )
    
    print(f"\nLightGBM Configuration (consistent with Cell 11):")
    print(f"  n_estimators: 100")
    print(f"  max_depth: 15")
    print(f"  learning_rate: 0.1")
    print(f"  num_leaves: 31")
    print(f"  min_child_samples: 20")
    print(f"  random_state: 42")
    print(f"  n_jobs: 1")
    print(f"  verbosity: -1")
    
    # 5-Fold Cross-Validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Store RMSE scores for refined model across all MICE datasets and CV folds
    refined_rmse_scores = []
    
    print(f"\nCross-Validation Setup:")
    print(f"  CV: {cv.n_splits}-fold KFold")
    print(f"  MICE Datasets: {M}")
    print(f"  Refined Model Features: {len(top_10_features)}")
    print(f"  Total evaluations: {M} datasets × {cv.n_splits} folds = {M * cv.n_splits}")
    
    print(f"\n{'='*60}")
    print("Training Refined Model Across MICE Datasets...")
    print(f"{'='*60}")
    
    # Main loop: iterate over MICE datasets
    for m_idx in range(M):
        print(f"\nProcessing MICE Dataset {m_idx+1}/{M}...")
        
        X_df = mice_X_dataframes[m_idx]
        y = mice_Y_datasets[m_idx]
        
        print(f"  Dataset shape: X {X_df.shape}, y {y.shape}")
        
        # Select only top 10 features
        available_top10 = [f for f in top_10_features if f in X_df.columns]
        
        if len(available_top10) < len(top_10_features):
            print(f"  [WARNING] Only {len(available_top10)}/{len(top_10_features)} top 10 features available")
            missing = set(top_10_features) - set(available_top10)
            print(f"    Missing: {missing}")
        
        if len(available_top10) == 0:
            print(f"  [ERROR] No top 10 features available, skipping this dataset")
            continue
        
        X_refined = X_df[available_top10].values
        
        print(f"  Refined model features: {len(available_top10)}")
        
        # Cross-validation (using neg_mean_squared_error, then convert to RMSE)
        neg_mse_scores = cross_val_score(
            lgb_model,
            X_refined,
            y,
            cv=cv,
            scoring='neg_mean_squared_error',
            n_jobs=1
        )
        
        # Convert to RMSE
        rmse_scores = np.sqrt(-neg_mse_scores)
        refined_rmse_scores.extend(rmse_scores)
        
        mean_rmse = np.mean(rmse_scores)
        std_rmse = np.std(rmse_scores)
        print(f"  Mean RMSE: {mean_rmse:.4f} (±{std_rmse:.4f})")
    
    # Calculate pooled RMSE for refined model
    refined_model_rmse = np.mean(refined_rmse_scores)
    refined_model_rmse_std = np.std(refined_rmse_scores)
    
    print(f"\n[OK] Refined model evaluation completed")
    print(f"  Pooled Mean RMSE: {refined_model_rmse:.4f} (±{refined_model_rmse_std:.4f})")
    print(f"  Total evaluations: {len(refined_rmse_scores)}")
    
    # ============================================================================
    # Task 3: Compare with Full Model
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: Compare with Full Model")
    print("="*60)
    
    # Load full model RMSE from Cell 11 results (RQ1 Block 4 - Full Model)
    benchmarking_file = 'RQ1_Incremental_RMSE_LightGBM.csv'
    try:
        rq1_results = pd.read_csv(benchmarking_file, encoding='utf-8')
        # Get Block 4 (Full Model) RMSE
        block4_row = rq1_results[rq1_results['Model Block'] == 'Block 4']
        if len(block4_row) > 0:
            full_model_rmse = block4_row.iloc[0]['Cumulative RMSE (Pooled Mean)']
            # Try to get std if available
            if 'RMSE Std' in rq1_results.columns:
                full_model_rmse_std = block4_row.iloc[0].get('RMSE Std', np.nan)
            else:
                full_model_rmse_std = np.nan
            print(f"[OK] Loaded Full Model RMSE from: {benchmarking_file}")
            print(f"  Full Model (Block 4) RMSE: {full_model_rmse:.4f}")
        else:
            print(f"[WARNING] Could not find Block 4 in {benchmarking_file}")
            print(f"  Using default value or calculating from Cell 11 results")
            # Fallback: use a reasonable default or calculate
            full_model_rmse = None
            full_model_rmse_std = None
    except FileNotFoundError:
        print(f"[WARNING] Could not find {benchmarking_file}")
        print(f"  Please run Cell 11 first to generate RQ1 results.")
        print(f"  Will use Refined Model RMSE only for comparison.")
        full_model_rmse = None
        full_model_rmse_std = None
    
    # If full model RMSE not available, we can still report refined model results
    if full_model_rmse is not None:
        # Calculate performance loss
        delta_rmse = refined_model_rmse - full_model_rmse
        performance_loss_pct = (delta_rmse / full_model_rmse) * 100 if full_model_rmse > 0 else 0
        
        # Create comparison table
        results_data = [
            {
                'Model Version': 'Full Model',
                'Number of Features': len(mice_X_dataframes[0].columns),
                'Mean RMSE': full_model_rmse,
                'RMSE Std': full_model_rmse_std if not np.isnan(full_model_rmse_std) else np.nan,
                'Difference': 0.0,
                'Performance_Loss_Pct': 0.0
            },
            {
                'Model Version': 'Refined Model',
                'Number of Features': len(top_10_features),
                'Mean RMSE': refined_model_rmse,
                'RMSE Std': refined_model_rmse_std,
                'Difference': delta_rmse,
                'Performance_Loss_Pct': performance_loss_pct
            }
        ]
        
        results_df = pd.DataFrame(results_data)
        
        # Print comparison table
        print("\n" + "="*100)
        print("RQ2: Refined Model vs Full Model Comparison")
        print("="*100)
        print(f"\n{'Model Version':<20} {'Number of Features':<20} {'Mean RMSE':<15} {'RMSE Std':<15} {'Difference (ΔRMSE)':<20} {'Performance Loss (%)':<20}")
        print("-"*100)
        
        for idx, row in results_df.iterrows():
            diff_str = f"{row['Difference']:+.4f}" if not np.isnan(row['Difference']) else "N/A"
            loss_str = f"{row['Performance_Loss_Pct']:+.2f}%" if not np.isnan(row['Performance_Loss_Pct']) else "N/A"
            rmse_std_str = f"±{row['RMSE Std']:.4f}" if not np.isnan(row['RMSE Std']) else "N/A"
            print(f"{row['Model Version']:<20} {row['Number of Features']:<20} {row['Mean RMSE']:>14.4f} {rmse_std_str:>14} {diff_str:>19} {loss_str:>19}")
        
        # Interpretation
        print("\n" + "="*100)
        print("Interpretation:")
        print("="*100)
        if delta_rmse > 0:
            print(f"  Refined Model has higher RMSE (+{delta_rmse:.4f}), indicating {performance_loss_pct:.2f}% performance loss.")
            print(f"  This suggests that the top 10 features alone cannot fully capture the predictive power of all features.")
        elif delta_rmse < 0:
            print(f"  Refined Model has lower RMSE ({delta_rmse:.4f}), indicating improved performance.")
            print(f"  This suggests that the top 10 features are sufficient and other features may add noise.")
        else:
            print(f"  Refined Model has similar RMSE, indicating that top 10 features are sufficient.")
        
        # Save results
        results_filename = 'RQ2_Refined_Model_LightGBM.csv'
        results_df.to_csv(results_filename, index=False, encoding='utf-8')
        print(f"\n[OK] Results saved: {results_filename}")
    else:
        # Only refined model results available
        results_data = [
            {
                'Model Version': 'Refined Model',
                'Number of Features': len(top_10_features),
                'Mean RMSE': refined_model_rmse,
                'RMSE Std': refined_model_rmse_std,
                'Difference': np.nan,
                'Performance_Loss_Pct': np.nan
            }
        ]
        results_df = pd.DataFrame(results_data)
        
        print("\n[INFO] Full Model RMSE not available. Showing Refined Model results only:")
        print(f"  Refined Model RMSE: {refined_model_rmse:.4f} (±{refined_model_rmse_std:.4f})")
        
        results_filename = 'RQ2_Refined_Model_LightGBM.csv'
        results_df.to_csv(results_filename, index=False, encoding='utf-8')
        print(f"\n[OK] Results saved: {results_filename}")
    
    print(f"\n{'='*60}")
    print("[OK] RQ2 Refined Model Comparison completed!")
    print(f"  Processed {M} MICE datasets")
    print(f"  Evaluated Refined Model with {len(top_10_features)} features")
    print(f"  Generated comparison table")
    print("="*60)

else:
    print("\n[ERROR] Cannot proceed:")
    if len(mice_X_datasets) != M or len(mice_Y_datasets) != M:
        print(f"  MICE datasets not loaded: {len(mice_X_datasets)} X, {len(mice_Y_datasets)} Y")
    if len(top_10_features) != 10:
        print(f"  Top 10 features not available: {len(top_10_features)} features found")
        print(f"  Please run Cell 12 first to generate SHAP importance rankings.")



Cell 13: RQ2 - Refined Model Comparison (LightGBM)

Task 1: Load Data and SHAP Rankings
[OK] Loaded SHAP importance rankings from: Rubins_Rule_SHAP_LightGBM.csv
  Total features in rankings: 10

[OK] Extracted top 10 features:
  1. cf23p455
  2. Delta_Skills
  3. cw23p003
  4. C_Score
  5. O_Score
  6. Delta_Workload
  7. Delta_Autonomy
  8. N_Score
  9. A_Score
  10. ch23p004

[INFO] Loading MICE datasets...
[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment
  Found 5 X datasets and 5 Y datasets
  Found 5 X dataframes with feature names

Task 2: Train Refined Model (Top 10 Features)

LightGBM Configuration (consistent with Cell 11):
  n_estimators: 100
  max_depth: 15
  learning_rate: 0.1
  num_leaves: 31
  min_child_samples: 20
  random_state: 42
  n_jobs: 1
  verbosity: -1

Cross-Validation Setup:
  CV: 5-fold KFold
  MICE Datasets: 5
  Refined Model Features: 10
  Total evaluations: 5 datasets × 5 folds = 25

Training Refined Model Across MICE Datase

In [5]:
# Cell 14:  RQ3 - Subgroup Fairness Analysis (LightGBM)
# RQ3: Subgroup Fairness Analysis using LightGBM
print("="*60)
print("Cell 14: RQ3 - Subgroup Fairness Analysis (LightGBM)")
print("="*60)

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Check if LightGBM is available
try:
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    raise ImportError("LightGBM is required but could not be imported.")

# ============================================================================
# Task 1: Environment Verification & Data Loading
# ============================================================================
print("\n" + "="*60)
print("Task 1: Environment Verification & Data Loading")
print("="*60)

# Case A: Check if mice_X_datasets and mice_Y_datasets exist in environment
try:
    if 'mice_X_datasets' in globals() and 'mice_Y_datasets' in globals() and \
       mice_X_datasets is not None and mice_Y_datasets is not None and \
       len(mice_X_datasets) > 0 and len(mice_Y_datasets) > 0:
        print("[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment")
        print(f"  Found {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")
        M = len(mice_X_datasets)
        if 'mice_X_dataframes' not in globals() or mice_X_dataframes is None or len(mice_X_dataframes) == 0:
            print("[WARNING] mice_X_dataframes not found, will reload to preserve feature names")
            raise NameError("mice_X_dataframes missing")
        else:
            print(f"  Found {len(mice_X_dataframes)} X dataframes with feature names")
            data_loaded = True
    else:
        raise NameError("Variables not available")
except (NameError, AttributeError):
    # Case B: Reload from CSV files
    print("[INFO] Case B: Reloading MICE datasets from CSV files")
    data_loaded = False

if not data_loaded:
    M = 5
    mice_X_datasets = []
    mice_Y_datasets = []
    mice_X_dataframes = []

    for m in range(1, M + 1):
        X_file = f'LISS_Final_X_M{m}.csv'
        Y_file = f'LISS_Final_Y_M{m}.csv'
        
        try:
            X_m = pd.read_csv(X_file, encoding='utf-8', index_col=0)
            Y_m = pd.read_csv(Y_file, encoding='utf-8', index_col=0)
            
            # Align indices
            common_idx = X_m.index.intersection(Y_m.index)
            X_m = X_m.loc[common_idx]
            Y_m = Y_m.loc[common_idx]
            
            # Check for missing values
            X_missing = X_m.isna().sum().sum()
            Y_missing = Y_m.isna().sum().sum()
            
            if X_missing > 0 or Y_missing > 0:
                valid_mask = ~(X_m.isna().any(axis=1) | Y_m.isna().any(axis=1))
                X_m = X_m[valid_mask]
                Y_m = Y_m[valid_mask]
            
            # Extract target variable
            target_col = Y_m.columns[0]
            y_m = Y_m[target_col].values
            X_m_values = X_m.values
            
            mice_X_datasets.append(X_m_values)
            mice_Y_datasets.append(y_m)
            mice_X_dataframes.append(X_m.copy())
            
            print(f"[OK] M{m}: X shape {X_m_values.shape}, y shape {y_m.shape}, target: {target_col}")
            
        except FileNotFoundError as e:
            print(f"[ERROR] M{m}: Could not find {X_file} or {Y_file}")
            break
        except Exception as e:
            print(f"[ERROR] M{m}: Error loading data: {e}")
            break

    if len(mice_X_datasets) == M:
        print(f"\n[OK] Successfully loaded {M} MICE datasets from CSV files")
        print(f"  Total datasets: {len(mice_X_datasets)}")
        print(f"  Feature names preserved for subgroup analysis")
    else:
        print(f"\n[ERROR] Only loaded {len(mice_X_datasets)}/{M} datasets. Cannot proceed.")
        mice_X_datasets = []
        mice_Y_datasets = []
        mice_X_dataframes = []

# Verify data is available
if len(mice_X_datasets) == M and len(mice_Y_datasets) == M:
    # ============================================================================
    # Task 2: Define Model and Evaluation Metric
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Define Model and Evaluation Metric")
    print("="*60)
    
    # Define LightGBM Regressor
    lgb_model = lgb.LGBMRegressor(
        n_estimators=100,
        max_depth=15,
        learning_rate=0.1,
        num_leaves=31,
        min_child_samples=20,
        random_state=42,
        n_jobs=1,
        verbosity=-1
    )
    
    print(f"\nLightGBM Configuration:")
    print(f"  n_estimators: 100")
    print(f"  max_depth: 15")
    print(f"  learning_rate: 0.1")
    print(f"  num_leaves: 31")
    print(f"  min_child_samples: 20")
    print(f"  random_state: 42")
    print(f"  n_jobs: 1")
    print(f"  verbosity: -1")
    
    print(f"\nEvaluation Metric:")
    print(f"  MAE (Mean Absolute Error) - more interpretable for subgroup comparisons")
    
    # ============================================================================
    # Task 3: Define Subgroups
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: Define Subgroups")
    print("="*60)
    
    # Get feature names
    feature_names = mice_X_dataframes[0].columns.tolist()
    
    # Define subgroups using lambda functions
    subgroups = {}
    
    # Gender subgroups
    gender_subgroups = {
        'Gender_Male (Baseline)': {
            'condition': lambda df: (df.get('ch23p001_2.0', pd.Series([False]*len(df), index=df.index)) == 0) &
                                    (df.get('ch23p001_3.0', pd.Series([False]*len(df), index=df.index)) == 0),
            'description': 'Male (baseline)'
        },
        'Gender_Female': {
            'condition': lambda df: df.get('ch23p001_2.0', pd.Series([False]*len(df), index=df.index)) == 1,
            'description': 'Female (ch23p001_2.0)'
        },
        'Gender_Other': {
            'condition': lambda df: df.get('ch23p001_3.0', pd.Series([False]*len(df), index=df.index)) == 1,
            'description': 'Other (ch23p001_3.0)'
        }
    }
    
    # Income subgroups
    income_subgroups = {
        'Income_Q1 (Baseline)': {
            'condition': lambda df: (df.get('Income_Quartile_2.0', pd.Series([False]*len(df), index=df.index)) == 0) &
                                    (df.get('Income_Quartile_3.0', pd.Series([False]*len(df), index=df.index)) == 0) &
                                    (df.get('Income_Quartile_4.0', pd.Series([False]*len(df), index=df.index)) == 0),
            'description': 'Income Quartile 1 (baseline, lowest)'
        },
        'Income_Q2': {
            'condition': lambda df: df.get('Income_Quartile_2.0', pd.Series([False]*len(df), index=df.index)) == 1,
            'description': 'Income Quartile 2 (Income_Quartile_2.0)'
        },
        'Income_Q3': {
            'condition': lambda df: df.get('Income_Quartile_3.0', pd.Series([False]*len(df), index=df.index)) == 1,
            'description': 'Income Quartile 3 (Income_Quartile_3.0)'
        },
        'Income_Q4': {
            'condition': lambda df: df.get('Income_Quartile_4.0', pd.Series([False]*len(df), index=df.index)) == 1,
            'description': 'Income Quartile 4 (Income_Quartile_4.0, highest)'
        }
    }
    
    # Education subgroups
    education_subgroups = {
        'Education_Low (Baseline)': {
            'condition': lambda df: (df.get('Education_Recode_2.0', pd.Series([False]*len(df), index=df.index)) == 0) &
                                    (df.get('Education_Recode_3.0', pd.Series([False]*len(df), index=df.index)) == 0) &
                                    (df.get('Education_Recode_4.0', pd.Series([False]*len(df), index=df.index)) == 0),
            'description': 'Education Level 1 (baseline, lowest)'
        },
        'Education_Medium': {
            'condition': lambda df: df.get('Education_Recode_2.0', pd.Series([False]*len(df), index=df.index)) == 1,
            'description': 'Education Level 2 (Education_Recode_2.0)'
        },
        'Education_High': {
            'condition': lambda df: df.get('Education_Recode_3.0', pd.Series([False]*len(df), index=df.index)) == 1,
            'description': 'Education Level 3 (Education_Recode_3.0)'
        },
        'Education_Highest': {
            'condition': lambda df: df.get('Education_Recode_4.0', pd.Series([False]*len(df), index=df.index)) == 1,
            'description': 'Education Level 4 (Education_Recode_4.0, highest)'
        }
    }
    
    # Combine all subgroups
    subgroups.update(gender_subgroups)
    subgroups.update(income_subgroups)
    subgroups.update(education_subgroups)
    
    print(f"\nDefined {len(subgroups)} subgroups:")
    print(f"  Gender: {len(gender_subgroups)} subgroups")
    print(f"  Income: {len(income_subgroups)} subgroups")
    print(f"  Education: {len(education_subgroups)} subgroups")
    
    # ============================================================================
    # Task 4: Execute Subgroup Cross-Validation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 4: Execute Subgroup Cross-Validation")
    print("="*60)
    
    # 5-Fold Cross-Validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Store MAE for each subgroup across all MICE datasets and CV folds
    subgroup_mae = {subgroup_name: [] for subgroup_name in subgroups.keys()}
    
    print(f"\nCross-Validation Setup:")
    print(f"  CV: {cv.n_splits}-fold KFold")
    print(f"  MICE Datasets: {M}")
    print(f"  Total evaluations: {M} datasets × {cv.n_splits} folds = {M * cv.n_splits} per subgroup")
    
    print(f"\n{'='*60}")
    print("Computing Subgroup MAE Across MICE Datasets...")
    print(f"{'='*60}")
    
    # Main loop: iterate over MICE datasets
    for m_idx in range(M):
        print(f"\nProcessing MICE Dataset {m_idx+1}/{M}...")
        
        X = mice_X_datasets[m_idx]
        y = mice_Y_datasets[m_idx]
        X_df = mice_X_dataframes[m_idx]
        
        print(f"  Dataset shape: X {X.shape}, y {y.shape}")
        
        # Cross-validation loop
        for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X), 1):
            # Split data
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            X_test_df = X_df.iloc[test_idx]
            
            # Train model
            lgb_model.fit(X_train, y_train)
            
            # Predict on test set
            y_pred = lgb_model.predict(X_test)
            
            # Calculate MAE for each subgroup in this test fold
            for subgroup_name, subgroup_info in subgroups.items():
                # Get subgroup mask for test set
                try:
                    subgroup_mask = subgroup_info['condition'](X_test_df)
                    
                    if subgroup_mask.sum() > 0:  # Only calculate if subgroup has samples
                        y_test_subgroup = y_test[subgroup_mask]
                        y_pred_subgroup = y_pred[subgroup_mask]
                        mae = mean_absolute_error(y_test_subgroup, y_pred_subgroup)
                        subgroup_mae[subgroup_name].append(mae)
                except Exception as e:
                    # If subgroup condition fails, skip
                    pass
        
        print(f"  [OK] M{m_idx+1}: Completed {cv.n_splits} CV folds")
    
    print(f"\n[OK] Subgroup MAE computed for all {M} MICE datasets")
    
    # ============================================================================
    # Task 5: Results Pooling and Output
    # ============================================================================
    print("\n" + "="*60)
    print("Task 5: Results Pooling and Output")
    print("="*60)
    
    # Calculate pooled MAE (average across all MICE datasets and CV folds)
    pooled_mae = {}
    pooled_mae_std = {}
    pooled_mae_count = {}
    
    for subgroup_name in subgroups.keys():
        if len(subgroup_mae[subgroup_name]) > 0:
            pooled_mae[subgroup_name] = np.mean(subgroup_mae[subgroup_name])
            pooled_mae_std[subgroup_name] = np.std(subgroup_mae[subgroup_name])
            pooled_mae_count[subgroup_name] = len(subgroup_mae[subgroup_name])
        else:
            pooled_mae[subgroup_name] = np.nan
            pooled_mae_std[subgroup_name] = np.nan
            pooled_mae_count[subgroup_name] = 0
    
    # Create results table
    results_data = []
    
    # Gender comparison
    gender_baseline = 'Gender_Male (Baseline)'
    if gender_baseline in pooled_mae and not np.isnan(pooled_mae[gender_baseline]):
        baseline_mae_gender = pooled_mae[gender_baseline]
        for subgroup_name in gender_subgroups.keys():
            if subgroup_name in pooled_mae:
                mae = pooled_mae[subgroup_name]
                mae_std = pooled_mae_std[subgroup_name]
                count = pooled_mae_count[subgroup_name]
                if not np.isnan(mae):
                    diff_from_baseline = mae - baseline_mae_gender
                    bias_rate = (diff_from_baseline / baseline_mae_gender) * 100 if baseline_mae_gender > 0 else 0
                    results_data.append({
                        'Category': 'Gender',
                        'Subgroup': subgroups[subgroup_name]['description'],
                        'MAE': mae,
                        'MAE_Std': mae_std,
                        'N_Evaluations': count,
                        'Diff_from_Baseline': diff_from_baseline,
                        'Bias_Rate_Pct': bias_rate,
                        'Baseline_MAE': baseline_mae_gender
                    })
    
    # Income comparison
    income_baseline = 'Income_Q1 (Baseline)'
    if income_baseline in pooled_mae and not np.isnan(pooled_mae[income_baseline]):
        baseline_mae_income = pooled_mae[income_baseline]
        for subgroup_name in income_subgroups.keys():
            if subgroup_name in pooled_mae:
                mae = pooled_mae[subgroup_name]
                mae_std = pooled_mae_std[subgroup_name]
                count = pooled_mae_count[subgroup_name]
                if not np.isnan(mae):
                    diff_from_baseline = mae - baseline_mae_income
                    bias_rate = (diff_from_baseline / baseline_mae_income) * 100 if baseline_mae_income > 0 else 0
                    results_data.append({
                        'Category': 'Income',
                        'Subgroup': subgroups[subgroup_name]['description'],
                        'MAE': mae,
                        'MAE_Std': mae_std,
                        'N_Evaluations': count,
                        'Diff_from_Baseline': diff_from_baseline,
                        'Bias_Rate_Pct': bias_rate,
                        'Baseline_MAE': baseline_mae_income
                    })
    
    # Education comparison
    education_baseline = 'Education_Low (Baseline)'
    if education_baseline in pooled_mae and not np.isnan(pooled_mae[education_baseline]):
        baseline_mae_education = pooled_mae[education_baseline]
        for subgroup_name in education_subgroups.keys():
            if subgroup_name in pooled_mae:
                mae = pooled_mae[subgroup_name]
                mae_std = pooled_mae_std[subgroup_name]
                count = pooled_mae_count[subgroup_name]
                if not np.isnan(mae):
                    diff_from_baseline = mae - baseline_mae_education
                    bias_rate = (diff_from_baseline / baseline_mae_education) * 100 if baseline_mae_education > 0 else 0
                    results_data.append({
                        'Category': 'Education',
                        'Subgroup': subgroups[subgroup_name]['description'],
                        'MAE': mae,
                        'MAE_Std': mae_std,
                        'N_Evaluations': count,
                        'Diff_from_Baseline': diff_from_baseline,
                        'Bias_Rate_Pct': bias_rate,
                        'Baseline_MAE': baseline_mae_education
                    })
    
    # Create DataFrame
    results_df = pd.DataFrame(results_data)
    
    # Print results table
    print("\n" + "="*100)
    print("RQ3: Subgroup Fairness Analysis Results (MAE)")
    print("="*100)
    print(f"\n{'Category':<12} {'Subgroup':<40} {'MAE':<12} {'MAE_Std':<12} {'Diff_from_Baseline':<20} {'Bias_Rate_Pct':<15} {'N_Evaluations':<15}")
    print("-"*100)
    
    for idx, row in results_df.iterrows():
        diff_str = f"{row['Diff_from_Baseline']:+.4f}" if not np.isnan(row['Diff_from_Baseline']) else "N/A"
        bias_str = f"{row['Bias_Rate_Pct']:+.2f}%" if not np.isnan(row['Bias_Rate_Pct']) else "N/A"
        print(f"{row['Category']:<12} {row['Subgroup']:<40} {row['MAE']:>11.4f} {row['MAE_Std']:>11.4f} {diff_str:>19} {bias_str:>14} {row['N_Evaluations']:>14}")
    
    # Identify significant biases
    print("\n" + "="*100)
    print("Bias Analysis:")
    print("="*100)
    
    significant_bias_threshold = 5.0  # 5% difference threshold
    
    for category in ['Gender', 'Income', 'Education']:
        category_results = results_df[results_df['Category'] == category]
        if len(category_results) > 0:
            baseline_row = category_results[category_results['Subgroup'].str.contains('baseline', case=False)]
            if len(baseline_row) > 0:
                baseline_mae = baseline_row.iloc[0]['MAE']
                print(f"\n{category} (Baseline MAE: {baseline_mae:.4f}):")
                
                for idx, row in category_results.iterrows():
                    if 'baseline' not in row['Subgroup'].lower():
                        bias_pct = row['Bias_Rate_Pct']
                        if not np.isnan(bias_pct) and abs(bias_pct) > significant_bias_threshold:
                            direction = "overestimated" if row['Diff_from_Baseline'] > 0 else "underestimated"
                            print(f"  {row['Subgroup']:<40} MAE: {row['MAE']:.4f} (Bias: {bias_pct:+.2f}%) - {direction}")
                        elif not np.isnan(bias_pct):
                            print(f"  {row['Subgroup']:<40} MAE: {row['MAE']:.4f} (Bias: {bias_pct:+.2f}%) - similar to baseline")
    
    # Save results
    results_filename = 'RQ3_Subgroup_MAE_LightGBM.csv'
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"\n[OK] Results saved: {results_filename}")
    
    print(f"\n{'='*60}")
    print("[OK] RQ3 Subgroup Fairness Analysis completed!")
    print(f"  Processed {M} MICE datasets")
    print(f"  Analyzed {len(subgroups)} subgroups")
    print(f"  Generated subgroup fairness analysis table")
    print("="*60)

else:
    print("\n[ERROR] Cannot proceed: MICE datasets not loaded successfully.")
    print(f"  Expected {M} datasets, but loaded {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")



Cell 14: RQ3 - Subgroup Fairness Analysis (LightGBM)

Task 1: Environment Verification & Data Loading
[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment
  Found 5 X datasets and 5 Y datasets
  Found 5 X dataframes with feature names

Task 2: Define Model and Evaluation Metric

LightGBM Configuration:
  n_estimators: 100
  max_depth: 15
  learning_rate: 0.1
  num_leaves: 31
  min_child_samples: 20
  random_state: 42
  n_jobs: 1
  verbosity: -1

Evaluation Metric:
  MAE (Mean Absolute Error) - more interpretable for subgroup comparisons

Task 3: Define Subgroups

Defined 11 subgroups:
  Gender: 3 subgroups
  Income: 4 subgroups
  Education: 4 subgroups

Task 4: Execute Subgroup Cross-Validation

Cross-Validation Setup:
  CV: 5-fold KFold
  MICE Datasets: 5
  Total evaluations: 5 datasets × 5 folds = 25 per subgroup

Computing Subgroup MAE Across MICE Datasets...

Processing MICE Dataset 1/5...
  Dataset shape: X (12939, 22), y (12939,)
  [OK] M1: Completed

In [ ]:
# Cell 15:  Visualization - Prediction Performance & SHAP (LightGBM)
# Visualization: Prediction Performance & SHAP using LightGBM
print("="*60)
print("Cell 15: Visualization - Prediction Performance & SHAP (LightGBM)")
print("="*60)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
import lightgbm as lgb
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Check if LightGBM is available
try:
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False
    raise ImportError("LightGBM is required but could not be imported.")

# Set style (using matplotlib instead of seaborn)
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    try:
        plt.style.use('seaborn-whitegrid')
    except:
        plt.style.use('default')
        # Manually set whitegrid-like style
        plt.rcParams['axes.grid'] = True
        plt.rcParams['grid.alpha'] = 0.3
        plt.rcParams['axes.axisbelow'] = True
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (12, 8)

# ============================================================================
# Task 1: Environment Verification & Data Loading
# ============================================================================
print("\n" + "="*60)
print("Task 1: Environment Verification & Data Loading")
print("="*60)

# Case A: Check if mice_X_datasets exist in environment
try:
    if 'mice_X_datasets' in globals() and 'mice_Y_datasets' in globals() and \
       mice_X_datasets is not None and mice_Y_datasets is not None and \
       len(mice_X_datasets) > 0 and len(mice_Y_datasets) > 0:
        print("[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment")
        print(f"  Found {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")
        M = len(mice_X_datasets)
        if 'mice_X_dataframes' not in globals() or mice_X_dataframes is None or len(mice_X_dataframes) == 0:
            print("[WARNING] mice_X_dataframes not found, will reload to preserve feature names")
            raise NameError("mice_X_dataframes missing")
        else:
            print(f"  Found {len(mice_X_dataframes)} X dataframes with feature names")
            data_loaded = True
    else:
        raise NameError("Variables not available")
except (NameError, AttributeError):
    # Case B: Reload from CSV files
    print("[INFO] Case B: Reloading MICE datasets from CSV files")
    data_loaded = False

if not data_loaded:
    M = 5
    mice_X_datasets = []
    mice_Y_datasets = []
    mice_X_dataframes = []

    for m in range(1, M + 1):
        X_file = f'LISS_Final_X_M{m}.csv'
        Y_file = f'LISS_Final_Y_M{m}.csv'
        
        try:
            X_m = pd.read_csv(X_file, encoding='utf-8', index_col=0)
            Y_m = pd.read_csv(Y_file, encoding='utf-8', index_col=0)
            
            # Align indices
            common_idx = X_m.index.intersection(Y_m.index)
            X_m = X_m.loc[common_idx]
            Y_m = Y_m.loc[common_idx]
            
            # Check for missing values
            X_missing = X_m.isna().sum().sum()
            Y_missing = Y_m.isna().sum().sum()
            
            if X_missing > 0 or Y_missing > 0:
                valid_mask = ~(X_m.isna().any(axis=1) | Y_m.isna().any(axis=1))
                X_m = X_m[valid_mask]
                Y_m = Y_m[valid_mask]
            
            # Extract target variable
            target_col = Y_m.columns[0]
            y_m = Y_m[target_col].values
            X_m_values = X_m.values
            
            mice_X_datasets.append(X_m_values)
            mice_Y_datasets.append(y_m)
            mice_X_dataframes.append(X_m.copy())
            
            print(f"[OK] M{m}: X shape {X_m_values.shape}, y shape {y_m.shape}, target: {target_col}")
            
        except FileNotFoundError as e:
            print(f"[ERROR] M{m}: Could not find {X_file} or {Y_file}")
            break
        except Exception as e:
            print(f"[ERROR] M{m}: Error loading data: {e}")
            break

    if len(mice_X_datasets) == M:
        print(f"\n[OK] Successfully loaded {M} MICE datasets from CSV files")
    else:
        print(f"\n[ERROR] Only loaded {len(mice_X_datasets)}/{M} datasets. Cannot proceed.")
        mice_X_datasets = []
        mice_Y_datasets = []
        mice_X_dataframes = []

# Verify data is available
if len(mice_X_datasets) == M and len(mice_Y_datasets) == M:
    # ============================================================================
    # Task 2: Generate Pooled Predictions
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Generate Pooled Predictions")
    print("="*60)
    
    # Define LightGBM Regressor
    lgb_model = lgb.LGBMRegressor(
        n_estimators=100,
        max_depth=15,
        learning_rate=0.1,
        num_leaves=31,
        min_child_samples=20,
        random_state=42,
        n_jobs=1,
        verbosity=-1
    )
    
    # 5-Fold Cross-Validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Store all actual and predicted values across all MICE datasets
    pooled_actual = []
    pooled_predicted = []
    pooled_train_y = []  # For distribution check
    pooled_test_y = []   # For distribution check
    
    print(f"\nCross-Validation Setup:")
    print(f"  CV: {cv.n_splits}-fold KFold")
    print(f"  MICE Datasets: {M}")
    print(f"  Total predictions: {M} datasets × {cv.n_splits} folds")
    
    print(f"\n{'='*60}")
    print("Generating Predictions Across MICE Datasets...")
    print(f"{'='*60}")
    
    # Main loop: iterate over MICE datasets
    for m_idx in range(M):
        print(f"\nProcessing MICE Dataset {m_idx+1}/{M}...")
        
        X = mice_X_datasets[m_idx]
        y = mice_Y_datasets[m_idx]
        
        print(f"  Dataset shape: X {X.shape}, y {y.shape}")
        
        # Use cross_val_predict to get predictions
        y_pred = cross_val_predict(lgb_model, X, y, cv=cv, n_jobs=1)
        
        # Collect actual and predicted values
        pooled_actual.extend(y)
        pooled_predicted.extend(y_pred)
        
        # Collect train/test distributions (approximate)
        for train_idx, test_idx in cv.split(X):
            pooled_train_y.extend(y[train_idx])
            pooled_test_y.extend(y[test_idx])
        
        # Calculate metrics for this dataset
        rmse = np.sqrt(mean_squared_error(y, y_pred))
        r2 = r2_score(y, y_pred)
        print(f"  M{m_idx+1} RMSE: {rmse:.4f}, R²: {r2:.4f}")
    
    print(f"\n[OK] Predictions generated for all {M} MICE datasets")
    print(f"  Total predictions: {len(pooled_actual):,}")
    
    # ============================================================================
    # Task 3: Actual vs Predicted Plot
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: Actual vs Predicted Plot")
    print("="*60)
    
    # Filter data for visualization (limit to [-10, 10])
    valid_range = [-10, 10]
    valid_mask = (np.array(pooled_actual) >= valid_range[0]) & (np.array(pooled_actual) <= valid_range[1]) & \
                 (np.array(pooled_predicted) >= valid_range[0]) & (np.array(pooled_predicted) <= valid_range[1])
    
    actual_plot = np.array(pooled_actual)[valid_mask]
    predicted_plot = np.array(pooled_predicted)[valid_mask]
    n_outliers = len(pooled_actual) - len(actual_plot)
    
    print(f"  Points within range [{valid_range[0]}, {valid_range[1]}]: {len(actual_plot):,}")
    print(f"  Outliers removed: {n_outliers:,}")
    
    # Create plot
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.scatter(actual_plot, predicted_plot, alpha=0.3, s=20, edgecolors='black', linewidth=0.5)
    
    # Perfect fit line (y = x)
    ax.plot([valid_range[0], valid_range[1]], [valid_range[0], valid_range[1]], 
           'r--', linewidth=2, label='Perfect Fit (y=x)', zorder=1)
    
    # Calculate and plot regression line
    z = np.polyfit(actual_plot, predicted_plot, 1)
    p = np.poly1d(z)
    x_line = np.linspace(valid_range[0], valid_range[1], 100)
    ax.plot(x_line, p(x_line), 'b-', linewidth=2, label=f'Regression (slope={z[0]:.3f})', alpha=0.8)
    
    ax.set_xlim(valid_range)
    ax.set_ylim(valid_range)
    ax.set_xlabel('Actual Δ JobSatisfaction', fontsize=14, fontweight='bold')
    ax.set_ylabel('Predicted Δ JobSatisfaction', fontsize=14, fontweight='bold')
    ax.set_title('Actual vs Predicted Δ JobSatisfaction\nOutliers removed for visualization clarity', 
                fontsize=16, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left', fontsize=12)
    ax.set_aspect('equal', adjustable='box')
    
    plt.tight_layout()
    fig1_filename = 'Pooled_Actual_vs_Predicted_LightGBM.png'
    plt.savefig(fig1_filename, dpi=300, bbox_inches='tight')
    print(f"  [OK] Saved: {fig1_filename}")
    plt.close(fig)
    
    # ============================================================================
    # Task 4: Residual Plot
    # ============================================================================
    print("\n" + "="*60)
    print("Task 4: Residual Plot")
    print("="*60)
    
    # Calculate residuals
    residuals_plot = actual_plot - predicted_plot
    
    # Create plot
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.scatter(predicted_plot, residuals_plot, alpha=0.3, s=20, edgecolors='black', linewidth=0.5)
    
    # Zero residual line
    ax.axhline(y=0, color='r', linestyle='--', linewidth=2, label='Zero Residual', zorder=1)
    
    ax.set_xlim(valid_range)
    ax.set_ylim(valid_range)
    ax.set_xlabel('Predicted Δ JobSatisfaction', fontsize=14, fontweight='bold')
    ax.set_ylabel('Residuals (Actual - Predicted)', fontsize=14, fontweight='bold')
    ax.set_title('Residual Plot\nOutliers removed for visualization clarity', 
                fontsize=16, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right', fontsize=12)
    
    plt.tight_layout()
    fig2_filename = 'Pooled_Residual_Plot_LightGBM.png'
    plt.savefig(fig2_filename, dpi=300, bbox_inches='tight')
    print(f"  [OK] Saved: {fig2_filename}")
    plt.close(fig)
    
    # ============================================================================
    # Task 5: SHAP Summary Plot
    # ============================================================================
    print("\n" + "="*60)
    print("Task 5: SHAP Summary Plot")
    print("="*60)
    
    # Train model on first MICE dataset for SHAP
    X_shap = mice_X_datasets[0]
    y_shap = mice_Y_datasets[0]
    X_shap_df = mice_X_dataframes[0]
    feature_names = X_shap_df.columns.tolist()
    
    print(f"  Training model on M1 for SHAP calculation...")
    lgb_model.fit(X_shap, y_shap)
    print(f"  [OK] Model trained")
    
    # Calculate SHAP values (sample for efficiency)
    print(f"  Computing SHAP values (sampling 1000 instances for efficiency)...")
    sample_size = min(1000, len(X_shap))
    sample_idx = np.random.choice(len(X_shap), sample_size, replace=False)
    X_shap_sample = X_shap[sample_idx]
    X_shap_df_sample = X_shap_df.iloc[sample_idx]
    
    explainer = shap.TreeExplainer(lgb_model)
    shap_values = explainer.shap_values(X_shap_sample)
    
    print(f"  [OK] SHAP values computed: shape {shap_values.shape}")
    
    # Create SHAP Summary Plot
    print(f"  Generating SHAP Summary Plot...")
    fig, ax = plt.subplots(figsize=(12, 10))
    shap.summary_plot(shap_values, X_shap_df_sample, feature_names=feature_names, 
                     show=False, plot_size=(12, 10))
    plt.tight_layout()
    
    fig3_filename = 'SHAP_Summary_Plot_LightGBM.png'
    plt.savefig(fig3_filename, dpi=300, bbox_inches='tight')
    print(f"  [OK] Saved: {fig3_filename}")
    plt.close(fig)
    
    # ============================================================================
    # Task 6: SHAP Dependence Plot (Top 1 Feature)
    # ============================================================================
    print("\n" + "="*60)
    print("Task 6: SHAP Dependence Plot (Top 1 Feature)")
    print("="*60)
    
    # Get top 1 feature from SHAP importance
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    top_feature_idx = np.argmax(mean_abs_shap)
    top_feature_name = feature_names[top_feature_idx]
    
    print(f"  Top 1 Feature: {top_feature_name} (Mean Abs SHAP: {mean_abs_shap[top_feature_idx]:.6f})")
    
    # Create SHAP Dependence Plot
    print(f"  Generating SHAP Dependence Plot...")
    fig, ax = plt.subplots(figsize=(10, 8))
    shap.dependence_plot(
        top_feature_idx,
        shap_values,
        X_shap_df_sample,
        feature_names=feature_names,
        show=False,
        ax=ax
    )
    plt.tight_layout()
    
    fig4_filename = f'SHAP_Dependence_{top_feature_name}_LightGBM.png'
    plt.savefig(fig4_filename, dpi=300, bbox_inches='tight')
    print(f"  [OK] Saved: {fig4_filename}")
    plt.close(fig)
    
    # ============================================================================
    # Task 7: Target Distribution Check (Train vs Test KDE)
    # ============================================================================
    print("\n" + "="*60)
    print("Task 7: Target Distribution Check (Train vs Test KDE)")
    print("="*60)
    
    # Create KDE plot
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Plot train and test distributions
    from scipy import stats
    
    train_y_array = np.array(pooled_train_y)
    test_y_array = np.array(pooled_test_y)
    
    # Filter to valid range
    train_y_valid = train_y_array[(train_y_array >= -10) & (train_y_array <= 10)]
    test_y_valid = test_y_array[(test_y_array >= -10) & (test_y_array <= 10)]
    
    # KDE
    train_kde = stats.gaussian_kde(train_y_valid)
    test_kde = stats.gaussian_kde(test_y_valid)
    
    x_range = np.linspace(-10, 10, 200)
    train_density = train_kde(x_range)
    test_density = test_kde(x_range)
    
    ax.plot(x_range, train_density, 'b-', linewidth=2, label='Train Set', alpha=0.7)
    ax.fill_between(x_range, train_density, alpha=0.3, color='blue')
    
    ax.plot(x_range, test_density, 'r-', linewidth=2, label='Test Set', alpha=0.7)
    ax.fill_between(x_range, test_density, alpha=0.3, color='red')
    
    ax.set_xlabel('Δ JobSatisfaction', fontsize=14, fontweight='bold')
    ax.set_ylabel('Density', fontsize=14, fontweight='bold')
    ax.set_title('Target Distribution: Train vs Test (KDE)', fontsize=16, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right', fontsize=12)
    
    plt.tight_layout()
    fig5_filename = 'Target_Distribution_Train_vs_Test_LightGBM.png'
    plt.savefig(fig5_filename, dpi=300, bbox_inches='tight')
    print(f"  [OK] Saved: {fig5_filename}")
    plt.close(fig)
    
    # ============================================================================
    # Task 8: Output Summary
    # ============================================================================
    print("\n" + "="*60)
    print("Task 8: Output Summary")
    print("="*60)
    
    # Calculate overall metrics
    overall_rmse = np.sqrt(mean_squared_error(pooled_actual, pooled_predicted))
    overall_r2 = r2_score(pooled_actual, pooled_predicted)
    
    print(f"\nOverall Model Performance (Pooled across {M} MICE datasets):")
    print(f"  RMSE: {overall_rmse:.4f}")
    print(f"  R²: {overall_r2:.4f}")
    print(f"  Total predictions: {len(pooled_actual):,}")
    
    print(f"\nGenerated Visualization Files:")
    files = [
        ('Pooled_Actual_vs_Predicted_LightGBM.png', 'Actual vs Predicted Scatter Plot'),
        ('Pooled_Residual_Plot_LightGBM.png', 'Residual Plot'),
        ('SHAP_Summary_Plot_LightGBM.png', 'SHAP Summary Plot'),
        (f'SHAP_Dependence_{top_feature_name}_LightGBM.png', f'SHAP Dependence Plot ({top_feature_name})'),
        ('Target_Distribution_Train_vs_Test_LightGBM.png', 'Target Distribution Check (Train vs Test)')
    ]
    
    import os
    for filename, description in files:
        if os.path.exists(filename):
            file_size = os.path.getsize(filename) / (1024 * 1024)
            print(f"  {filename}")
            print(f"    Description: {description}")
            print(f"    Size: {file_size:.2f} MB")
    
    print(f"\n{'='*60}")
    print("[OK] Visualization completed!")
    print(f"  Processed {M} MICE datasets")
    print(f"  Generated 5 visualization figures")
    print("="*60)

else:
    print("\n[ERROR] Cannot proceed: MICE datasets not loaded successfully.")
    print(f"  Expected {M} datasets, but loaded {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")


Cell 15: Visualization - Prediction Performance & SHAP (LightGBM)


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Cell 16:  Appendix - Correlation Matrix Analysis
# Appendix: Correlation Matrix Analysis
print("="*60)
print("Cell 16: Appendix - Correlation Matrix Analysis")
print("="*60)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Set style (using matplotlib instead of seaborn)
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except:
    try:
        plt.style.use('seaborn-whitegrid')
    except:
        plt.style.use('default')
        # Manually set whitegrid-like style
        plt.rcParams['axes.grid'] = True
        plt.rcParams['grid.alpha'] = 0.3
        plt.rcParams['axes.axisbelow'] = True
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (14, 12)

# ============================================================================
# Task 1: Load MICE Datasets
# ============================================================================
print("\n" + "="*60)
print("Task 1: Load MICE Datasets")
print("="*60)

# Case A: Check if mice_X_datasets exist in environment
try:
    if 'mice_X_datasets' in globals() and 'mice_Y_datasets' in globals() and \
       mice_X_datasets is not None and mice_Y_datasets is not None and \
       len(mice_X_datasets) > 0 and len(mice_Y_datasets) > 0:
        print("[INFO] Case A: Using existing mice_X_datasets and mice_Y_datasets from environment")
        print(f"  Found {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")
        M = len(mice_X_datasets)
        if 'mice_X_dataframes' not in globals() or mice_X_dataframes is None or len(mice_X_dataframes) == 0:
            print("[WARNING] mice_X_dataframes not found, will reload to preserve feature names")
            raise NameError("mice_X_dataframes missing")
        else:
            print(f"  Found {len(mice_X_dataframes)} X dataframes with feature names")
            data_loaded = True
    else:
        raise NameError("Variables not available")
except (NameError, AttributeError):
    # Case B: Reload from CSV files
    print("[INFO] Case B: Reloading MICE datasets from CSV files")
    data_loaded = False

if not data_loaded:
    M = 5
    mice_X_datasets = []
    mice_Y_datasets = []
    mice_X_dataframes = []

    for m in range(1, M + 1):
        X_file = f'LISS_Final_X_M{m}.csv'
        Y_file = f'LISS_Final_Y_M{m}.csv'
        
        try:
            X_m = pd.read_csv(X_file, encoding='utf-8', index_col=0)
            Y_m = pd.read_csv(Y_file, encoding='utf-8', index_col=0)
            
            # Align indices
            common_idx = X_m.index.intersection(Y_m.index)
            X_m = X_m.loc[common_idx]
            Y_m = Y_m.loc[common_idx]
            
            # Check for missing values
            X_missing = X_m.isna().sum().sum()
            Y_missing = Y_m.isna().sum().sum()
            
            if X_missing > 0 or Y_missing > 0:
                valid_mask = ~(X_m.isna().any(axis=1) | Y_m.isna().any(axis=1))
                X_m = X_m[valid_mask]
                Y_m = Y_m[valid_mask]
            
            # Extract target variable
            target_col = Y_m.columns[0]
            y_m = Y_m[target_col].values
            X_m_values = X_m.values
            
            mice_X_datasets.append(X_m_values)
            mice_Y_datasets.append(y_m)
            mice_X_dataframes.append(X_m.copy())
            
            print(f"[OK] M{m}: X shape {X_m_values.shape}, y shape {y_m.shape}, target: {target_col}")
            
        except FileNotFoundError as e:
            print(f"[ERROR] M{m}: Could not find {X_file} or {Y_file}")
            break
        except Exception as e:
            print(f"[ERROR] M{m}: Error loading data: {e}")
            break

    if len(mice_X_datasets) == M:
        print(f"\n[OK] Successfully loaded {M} MICE datasets from CSV files")
    else:
        print(f"\n[ERROR] Only loaded {len(mice_X_datasets)}/{M} datasets. Cannot proceed.")
        mice_X_datasets = []
        mice_Y_datasets = []
        mice_X_dataframes = []

# Verify data is available
if len(mice_X_datasets) == M and len(mice_Y_datasets) == M:
    # ============================================================================
    # Task 2: Select Continuous Variables
    # ============================================================================
    print("\n" + "="*60)
    print("Task 2: Select Continuous Variables")
    print("="*60)
    
    # Get feature names
    feature_names = mice_X_dataframes[0].columns.tolist()
    
    # Identify continuous variables (exclude OHE dummy variables)
    # Continuous variables: Big Five scores, Delta variables, baseline controls
    continuous_features = []
    
    # Big Five scores
    big_five = ['N_Score', 'E_Score', 'C_Score', 'A_Score', 'O_Score']
    continuous_features.extend([f for f in big_five if f in feature_names])
    
    # Delta variables
    delta_vars = ['Delta_Autonomy', 'Delta_Workload', 'Delta_Skills', 'Delta_RemoteHours', 
                   'Delta_WFC_Proxy', 'Delta_Health_Hindrance']
    continuous_features.extend([f for f in delta_vars if f in feature_names])
    
    # Baseline controls
    baseline_controls = ['cw23p003', 'cf23p455', 'ch23p004']  # Age, Children Count, Self-rated Health
    continuous_features.extend([f for f in baseline_controls if f in feature_names])
    
    # Target variable
    target_name = 'Delta_JobSatisfaction'
    
    print(f"\nSelected Continuous Variables:")
    print(f"  Big Five Scores: {len([f for f in big_five if f in continuous_features])}/{len(big_five)}")
    print(f"  Delta Variables: {len([f for f in delta_vars if f in continuous_features])}/{len(delta_vars)}")
    print(f"  Baseline Controls: {len([f for f in baseline_controls if f in continuous_features])}/{len(baseline_controls)}")
    print(f"  Total continuous features: {len(continuous_features)}")
    print(f"  Target variable: {target_name}")
    
    # ============================================================================
    # Task 3: Calculate Pooled Pearson Correlation
    # ============================================================================
    print("\n" + "="*60)
    print("Task 3: Calculate Pooled Pearson Correlation")
    print("="*60)
    
    # Store correlation matrices from each MICE dataset
    correlation_matrices = []
    
    for m_idx in range(M):
        print(f"\nProcessing MICE Dataset {m_idx+1}/{M}...")
        
        X_df = mice_X_dataframes[m_idx]
        y = mice_Y_datasets[m_idx]
        
        # Select continuous features
        available_continuous = [f for f in continuous_features if f in X_df.columns]
        
        if len(available_continuous) == 0:
            print(f"  [WARNING] No continuous features available, skipping")
            continue
        
        # Create DataFrame with continuous features and target
        data_for_corr = X_df[available_continuous].copy()
        data_for_corr[target_name] = y
        
        # Calculate correlation matrix
        corr_matrix = data_for_corr.corr(method='pearson')
        correlation_matrices.append(corr_matrix)
        
        print(f"  [OK] Correlation matrix computed: shape {corr_matrix.shape}")
    
    # Pool correlation matrices (average across MICE datasets)
    if len(correlation_matrices) > 0:
        # Align all correlation matrices to have the same columns/rows
        all_features = set()
        for corr_mat in correlation_matrices:
            all_features.update(corr_mat.columns)
        
        all_features = sorted(list(all_features))
        
        # Reindex all matrices to have the same structure
        aligned_matrices = []
        for corr_mat in correlation_matrices:
            aligned = corr_mat.reindex(index=all_features, columns=all_features)
            aligned_matrices.append(aligned)
        
        # Average across MICE datasets
        pooled_corr = pd.DataFrame(np.mean([m.values for m in aligned_matrices], axis=0),
                                   index=all_features, columns=all_features)
        
        print(f"\n[OK] Pooled correlation matrix computed: shape {pooled_corr.shape}")
        print(f"  Features: {len(all_features)}")
        
        # ============================================================================
        # Task 4: Visualize Correlation Matrix
        # ============================================================================
        print("\n" + "="*60)
        print("Task 4: Visualize Correlation Matrix")
        print("="*60)
        
        # Create heatmap
        fig, ax = plt.subplots(figsize=(14, 12))
        
        # Create mask for upper triangle (optional, to show only lower triangle)
        mask = np.triu(np.ones_like(pooled_corr, dtype=bool), k=1)
        
        # Apply mask to correlation matrix (set upper triangle to NaN)
        corr_masked = pooled_corr.copy()
        corr_masked[mask] = np.nan
        
        # Plot heatmap using matplotlib imshow
        im = ax.imshow(corr_masked, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1, 
                      interpolation='nearest')
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=ax, shrink=0.8)
        cbar.set_label('Correlation', rotation=270, labelpad=20)
        
        # Add annotations
        for i in range(len(corr_masked.index)):
            for j in range(len(corr_masked.columns)):
                if not mask[i, j]:  # Only annotate lower triangle
                    value = corr_masked.iloc[i, j]
                    if not np.isnan(value):
                        text = ax.text(j, i, f'{value:.2f}',
                                     ha="center", va="center", color="black", fontsize=8)
        
        # Set ticks and labels
        ax.set_xticks(np.arange(len(corr_masked.columns)))
        ax.set_yticks(np.arange(len(corr_masked.index)))
        ax.set_xticklabels(corr_masked.columns, rotation=45, ha='right', fontsize=10)
        ax.set_yticklabels(corr_masked.index, fontsize=10)
        
        # Add grid lines
        ax.set_xticks(np.arange(len(corr_masked.columns)) - 0.5, minor=True)
        ax.set_yticks(np.arange(len(corr_masked.index)) - 0.5, minor=True)
        ax.grid(which="minor", color="gray", linestyle='-', linewidth=0.5)
        
        ax.set_title('Pooled Pearson Correlation Matrix\n(Continuous Variables & Target)', 
                    fontsize=16, fontweight='bold', pad=20)
        
        plt.tight_layout()
        fig_filename = 'Correlation_Matrix_LightGBM.png'
        plt.savefig(fig_filename, dpi=300, bbox_inches='tight')
        print(f"  [OK] Saved: {fig_filename}")
        plt.close(fig)
        
        # ============================================================================
        # Task 5: Extract and Report Key Correlations
        # ============================================================================
        print("\n" + "="*60)
        print("Task 5: Extract and Report Key Correlations")
        print("="*60)
        
        # Get correlations with target variable
        if target_name in pooled_corr.columns:
            target_correlations = pooled_corr[target_name].drop(target_name).sort_values(ascending=False, key=abs)
            
            print(f"\nTop 10 Features Most Correlated with {target_name}:")
            print("-"*60)
            print(f"{'Feature':<35} {'Correlation':<15}")
            print("-"*60)
            
            for feat_name, corr_value in target_correlations.head(10).items():
                print(f"{feat_name:<35} {corr_value:>14.4f}")
            
            # Save correlation with target
            target_corr_df = pd.DataFrame({
                'Feature': target_correlations.index,
                'Correlation_with_Target': target_correlations.values
            })
            target_corr_df = target_corr_df.sort_values('Correlation_with_Target', ascending=False, key=lambda x: x.abs())
            
            target_corr_filename = 'Correlation_with_Target_LightGBM.csv'
            target_corr_df.to_csv(target_corr_filename, index=False, encoding='utf-8')
            print(f"\n[OK] Correlations with target saved: {target_corr_filename}")
        
        # Save full correlation matrix
        corr_matrix_filename = 'Pooled_Correlation_Matrix_LightGBM.csv'
        pooled_corr.to_csv(corr_matrix_filename, index=True, encoding='utf-8')
        print(f"[OK] Full correlation matrix saved: {corr_matrix_filename}")
        
        # ============================================================================
        # Task 6: Output Summary
        # ============================================================================
        print("\n" + "="*60)
        print("Task 6: Output Summary")
        print("="*60)
        
        print(f"\n[OK] Correlation Matrix Analysis completed!")
        print(f"  Processed {M} MICE datasets")
        print(f"  Analyzed {len(continuous_features)} continuous features")
        print(f"  Generated correlation heatmap")
        print(f"  Extracted correlations with target variable")
        
        import os
        files = [
            ('Correlation_Matrix_LightGBM.png', 'Correlation Heatmap'),
            ('Pooled_Correlation_Matrix_LightGBM.csv', 'Full Correlation Matrix'),
            ('Correlation_with_Target_LightGBM.csv', 'Correlations with Target')
        ]
        
        print(f"\nGenerated Files:")
        for filename, description in files:
            if os.path.exists(filename):
                if filename.endswith('.png'):
                    file_size = os.path.getsize(filename) / (1024 * 1024)
                    print(f"  {filename}")
                    print(f"    Description: {description}")
                    print(f"    Size: {file_size:.2f} MB")
                else:
                    file_size = os.path.getsize(filename) / 1024
                    print(f"  {filename}")
                    print(f"    Description: {description}")
                    print(f"    Size: {file_size:.2f} KB")
        
        print(f"\n{'='*60}")
        print("[OK] Appendix - Correlation Matrix Analysis completed!")
        print("="*60)
    else:
        print("\n[ERROR] Could not compute correlation matrices from MICE datasets")

else:
    print("\n[ERROR] Cannot proceed: MICE datasets not loaded successfully.")
    print(f"  Expected {M} datasets, but loaded {len(mice_X_datasets)} X datasets and {len(mice_Y_datasets)} Y datasets")

